# Coursework 3: Reed-Solomon Codes
## Group 17

[√] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

## 3.1 RS Codes on $\mathbb{F}_p$ (50%)

### 3.1.1 RS Codes on $\mathbb{F}_{17}$

1. In $\mathbb{F}_{17}$, find a primitive element $\alpha$. 
2. Let $\mathcal{C} \subset \mathbb{F}_{17}^{16}$ be an RS code with parameters $[16,k,7]$. What is the value of $k$? 
3. Design and implement an encoding function `RSNum_Enc` that encodes a message $\boldsymbol{m} \in \mathbb{F}_{17}^k$ to a codeword $\boldsymbol{c} \in \mathcal{C}$. 
4. Let $\boldsymbol{y} = \boldsymbol{c} + \boldsymbol{e}$ be the received word where the channel introduces $t = \lfloor \frac{d-1}{2}\rfloor$ errors in random places. Design and implement a function `RSNum_Channel` for this. Julia function `StatsBase > Sample` can be helpful in randomly chose error positions.
5. Design and implement a function `RSNum_Syndrome` to calculate the syndrome vector and the syndrome polynomial from the received word $\boldsymbol{y}$. 
6. Design and implement a function `RSNum_KeyPolys` to find error locator polynomial $L(z)$ and error evaluator polynomial $E(z)$ from the syndrome polynomial $S(z)$. 
7. Design and implement a function `RSNum_Err` to find the estimated error vector $\hat{\boldsymbol{e}}$ and the estimated codeword $\hat{\boldsymbol{c}}$.
8. Design and implement a function `RSNum_Message` to find the estimated message $\hat{\boldsymbol{m}}$. 
9. Implement a function `RSNum_Dec` which uses the functions in Steps 5-8 as sub-routines for decoding.  

Provide necessary documentation.

## 1.  Find a primitive element
Use the function `find_primitive` to find all the primitive elements in $\mathbb{F}_{17}$, which forms a set. Then, choose one primitive element randomly from the set, denoted as 'a'.

(Primitive Element: Order of an element is the minimum t that satisfies $a^t=1$ in $\mathbb{F}_{p}$. If $t=p-1$, this element is a primitive element.)

In [56]:
using StatsBase


"""
    find_primitive(p)

    Find the set of primitive elements for p
"""
function find_primitive(p)
    PE = Int64[]  # Initialize the set to contain all the primitive elements
    for i in 2:p-1  # Ergodic process to find all the primitive elements
        ord = 1
        n = i
        while n != 1  # Run until r is 1
            ord = ord + 1
            n = mod(i*n,p)
        end
        if ord == p - 1   # If the order is p-1, it is a primitive element
            push!(PE,i);
        end
    end
    return PE
end



"""
    a_generator(p)

    Randomly choose a primitive element from the set
"""
function a_generator(p)
    PE_set = find_primitive(p)  # Find the set of primitive element for p
    
    index = sample(collect(1:size(PE_set,1)), 1, replace=false)  # Randomly generate the index
    a = PE_set[index[1]]  # a_index is an 1-element vector, use index[1] to get its value

    println("1.  The chosen primitive element a = ",a,"\n")
    return a
end

a_generator

In [57]:
# Test
p = 17
println("The primitive element set is ", find_primitive(p),"\n")
a_generator(p)
nothing

The primitive element set is [3, 5, 6, 7, 10, 11, 12, 14]

1.  The chosen primitive element a = 11



## 2. Find the value of k
The relationship between $[n,k,d]$ and p is: $n=p-1$, $k=n-d+1$.

That is, in the given finite filed $\mathbb{F}_{17}^{16}$, $n = p - 1 = 16$. 

And if one of $k$ and $d$ is known, the other one can be found via $k=n-d+1$. Hence, for $d = 7$, $k = n - d + 1 = 16 - 7 + 1 = 10$.

In [58]:
"""
    k_generator(p,d)

    Generator k
"""
function k_generator(p,d)
    k = (p - 1) - d + 1  # k = n - d + 1, n = p - 1
    return k
end

k_generator

In [59]:
# Test
p = 17
d = 7
@show p
@show d
println("k = ", k_generator(p,d))
nothing

p = 17
d = 7
k = 10


## 3. Generate message $m$ and codeword $c$
Message $m$ is a polynomial with a degree no larger than $k-1$. Therefore, we can simply let the degree of m equals to $k-1$ and generate $k$ coefficients randomly, $m{\left(x\right)}=m_0+m_1x+m_2x^2+\ldots+m_{k-1}x^{k-1}$

After choosing the primitive element a, vector A can be formed as [1, $a$, $a^2$, ... , $a^{p-2}$]. Apply the evaluation mapping, map polynomial m to vector $c$, the codeword $c$ is determined.

In [60]:
## Abandoned method, reason is explained bellow this block. Just for demonstration
## !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

using Polynomials


"""
    RSNum_Enc_using_Polynomials_Abandoned(p,k,a)

    Encode the message.
"""
function RSNum_Enc_using_Polynomials_Abandoned(p,k,a)
    # Initialization
    m = Int64[]  # message polynomial (coefficients)
    c = zeros(Int64,p-1)  # codeword

    # Generate the polynomial m(x)
    for i in 1:k
        push!(m,rand(0:p-1)) # Randomly generate coefficients in the finite field for m
    end
    println("Message m is ",m,"\n")

    # Generate codeword c
    m_x = Polynomial(m)  # Generate the polynomial m(x)
    for i in 1:p-1
        a_i = powermod(a,i-1,p)  # mod(a^(i-1),p), calculate the ith element in A ([1,a,a^2,...,a^(p-2)])
        c[i] = trunc(Int,mod(m_x(a_i),p))  # substituting every the ith element in A to calculate the ith element of codeword c
    end
    println("Codeword c is ",vec(c),"\n")
    return c
end

RSNum_Enc_using_Polynomials_Abandoned

In [61]:
# Test
p = 17
d = 7
k = k_generator(p,d)
a = a_generator(p)
RSNum_Enc_using_Polynomials_Abandoned(p,k,a)
nothing

1.  The chosen primitive element a = 6

Message m is [13

, 10, 16, 11, 7, 8, 11, 10, 12, 3]

Codeword c is [16, 9, 5, 14, 5, 12, 6, 1, 0, 0, 3, 11, 5, 10, 7, 2]



However, the method above has a problem.

When $k$ is large / the degree of $m(x)$ is large, some results may go beyond the bound, e.g. for $k=49$, $x^{48}$ cannot be calculated accurately, in this form: '4.564684651865564e48'.

To solve this problem, the generator matrix $G$ is used to find the codeword, the process can be written as $$m*G=c$$  

In [62]:
# The correct one

"""
    RSNum_Enc(p,k,a)

    Encode the message.
"""
function RSNum_Enc(p,k,a)
    # Initialization
    m = Int64[]  # message polynomial (coefficients)
    G = zeros(Int64,k,p-1)  # generator matrix
    c = zeros(Int64,p-1)  # codeword
    
    # Generate the polynomial m(x)
    for i in 1:k
        push!(m,rand(0:p-1)) # Randomly generate coefficients in the finite field for m
    end
    println("2.  Message m is ",m,"\n")

    # Generate the generator matrix G
    for i in 1:k
        for j in 1:p-1
            G[i,j] = powermod(a,(i-1)*(j-1),p)  # G[i,j] = a^((i-1)(j-1)) mod p
        end
    end

    # Generate codeword c
    c = (mod.(m'*G,p))'
    println("3.  Codeword c is ",vec(c),"\n")
    return m, c
end

RSNum_Enc

In [63]:
# Test
p = 17
d = 7
k = k_generator(p,d)
a = a_generator(p)
RSNum_Enc(p,k,a)
nothing

1.  The chosen primitive element a = 3



2.  Message m is [9, 13, 7, 9, 16, 7, 13, 5, 6, 8]

3.  Codeword c is [8, 1, 5, 5, 6, 10, 0, 13, 9, 2, 7, 12, 16, 15, 1, 0]



## 4. Received word
The received word $r=c+e$ is got by introducing random errors in random positions to c.

In [64]:
using StatsBase


"""
    RSNum_Channel(p,d,c,t)

    Add noise to the codeword.
"""
function RSNum_Channel(p,d,c,t)
    index = sample(collect(1:length(c)), t, replace=false); # Randomly choose t indexes using sample function, which is the position for errors
    println("4.  Error position is ",sort(index,by=abs),"\n")  # show the sorted error position

    e = zeros(Int64,p-1)  # Initialize the error vector
    r = c'  # Initialize the received word
    for i in 1:t
        e[index[i]] = rand(1:p-1)   # Randomly generate the error 
        r[index[i]] = mod(c[index[i]] + e[index[i]], p)  # Add the random errors to the codeword to form the received word, r = c + e
    end
    println("5.  Error vector is ",e,"\n")
    println("6.  Received word is ",vec(r),"\n")
    return r
end

RSNum_Channel

In [65]:
# Test
p = 17
d = 7
k = k_generator(p,d)
t = floor(Int,(d-1)/2)
a = a_generator(p)
m,c = RSNum_Enc(p,k,a)
RSNum_Channel(p,d,c,t)
nothing

1.  The chosen primitive element a = 11

2.  Message m is [5, 10, 10, 6, 14, 9, 5, 4, 12, 4]

3.  Codeword c is [11, 1, 15, 4, 0, 3, 10, 14, 13, 13, 14, 16, 

15, 9, 7, 3]

4.  Error position is [4, 5, 10]

5.  Error vector is [0, 0, 0, 2, 9, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0]

6.  Received word is 

[11, 1, 15, 6, 9, 3, 10, 14, 13, 15, 14, 16, 15, 9, 7, 3]



### 5. Generate $S\left(z\right)$ and $s$
The syndrome polynomial $S\left(z\right)$ can be generated as $S\left(z\right)=\sum_{j=0}^{n-k-1}{s_jz^j}$, where $s=rH^T$, $r$ is the received message, and $H$ is the parity-check matrix whose every entry equals to $\alpha^{i\left(j-1\right)}$ and size is $\left(n-k\right)\times n$.

In [66]:
using Polynomials


"""
    RSNum_Syndrome(p,n,k,a,r)

    Generate the Syndrome Polynomial
"""
function RSNum_Syndrome(p,n,k,a,r)
    H = zeros(Int64,n-k,n)  # Initialize the parity-check matrix
    for i in 1:n-k
        for j in 1:n
            H[i,j] = powermod(a,i*(j-1),p)  # parity-check matrix H[i,j] = a^(i*(j-1)) mod p
        end
    end
    s_1 = r * H'

    # Apply modulation and change the type of s_1 into Int64
    S = zeros(Int64,n-k)
    for i in 1:n-k
        S[i] = mod(trunc(Int,s_1[i]),p)  # change the type of s_1 and apply modulation to each element of s_1
    end

    println("7.  Syndrome polynomial S(z) = ",Polynomial(S,:z),"\n")  # show the syndrome polynomial
    return S  # return the coefficients of syndrome polynomial
end

RSNum_Syndrome

In [67]:
# Test
p = 17
n = 16
d = 7
t = floor(Int,(d-1)/2)
k = k_generator(p,d)
a = a_generator(p)
m,c = RSNum_Enc(p,k,a)
r = RSNum_Channel(p,d,c,t)
RSNum_Syndrome(p,n,k,a,r)
nothing

1.  The chosen primitive element a = 5

2.  Message m is [11

, 14, 2, 3, 7, 6, 8, 15, 1, 0]

3.  Codeword c is [16, 13, 1, 2, 1, 6, 3, 8, 8, 15, 6, 6, 0, 13, 10, 0]

4.  Error position is [9, 13, 16]

5.  Error vector is [0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 12, 0, 0, 16]

6.  Received word is [

16, 13, 1, 2, 1, 6, 3, 8, 7, 15, 6, 6, 12, 13, 10, 16]

7.  Syndrome polynomial S(z) = 8 + 6*z + z^2 + 7*z^3 + 4*z^4 + 12*z^5



## 6. Generate $L\left(z\right)$ and $E\left(z\right)$
Apply  Euclidean algorithm to $z^{d-1}$ and $S\left(z\right)$

$$z^{d-1}=q_1\left(z\right)S\left(z\right)+r_1\left(z\right)$$
$$S\left(z\right)=q_2\left(z\right)r_1\left(z\right)+r_2\left(z\right)$$
$$...$$
$$r_{l-2}=q_l\left(z\right)r_{l-1}\left(z\right)+r_l\left(z\right)$$

where $deg(r_l\left(z\right))$≤ $t$-1

By Bezout's Identity, one has,
$$r_l\left(z\right)=a\left(z\right)S\left(z\right)+b\left(z\right)z^{d-1}\equiv a\left(z\right)S\left(z\right)mod {z^{d-1}}$$

Let $c$ be the leading coefficient of the polynomial $a\left(z\right)$, i.e., $c^{-1}a\left(z\right)$ is a monic polynomial. Then, $$L\left(z\right)=c^{-1}a\left(z\right)$$ $$E\left(z\right)=c^{-1}r_l\left(z\right)$$

In the following block, all the functions come from previous courseworks.

In [68]:
"""
    isprime(n)

    Used to determine whether the input is a prime number 
    Return 'true' if it is a prime number, return 'false' if not
"""
function isprime(n)
    if n < 2
        return false    
    elseif n == 2
        return true
    # Check all even numbers
    elseif n % 2 == 0
        return false
    end
    # Consider all odd numbers
    for i in collect(3:2:√n) # only need to check i < √n
        if n % i == 0 
            return false
        end
    end
    # if no f_xs were found
    true  
end



"""
    getdeg(A)

    Determine the degree of polynomial A. On the right is the coefficient of the largest term
    e.g. 3x^2+2x+1 then input A = [1 2 3]
"""
function getdeg(A)
    n = length(A)

    for i in 1:length(A)
        if A[n] == 0 # Determine whether the last bit is 0, if yes, reduce 1 to the length of A
            n -= 1; # Judge the next one in the next loop
            if n == 0 # if all of A equal to 0, then return degree 0
                return 0
            end
        else
            return n-1 # degree starts from 0, and the number of items starts from 1, so return n-1
        end
    end
end



"""
    fractionmod(molecular,denominator,p)

    Apply modulation to fraction numbers
"""
function fractionmod(molecular,denominator,p) # input the molecular and denominator of the fraction, then get the mod
    n=0 
    judgement = true #make judgement whether jump out the loop
    while judgement
        n += 1
        judgement = (mod(molecular,p) != mod(denominator*n,p)) 
        #Judge how many n can make the polynomial hold, they molecular%p equal to denominator module p, return n
    end
    return n
end



"""
    polydiv(fx,gx,p)

    Polynomial division, Find the remainder and quotient of fx/gx in p-domain
"""
function polydiv(fx,gx,p)
    if isprime(p)  # decide it's prime or not, if not, then return error
        fx = mod.(fx,p)  # Take the modulus of fx to ensure that coefficients of fx is in Fp
        gx = mod.(gx,p)  # Take the modulus of gx to ensure that coefficients of gx is in Fp
        deg_fx = getdeg(fx)  # get the degree of fx and gx
        deg_gx = getdeg(gx)
        
        fx = fx[1:deg_fx+1]  # Discard the extra 0 in fx, e.g. [1 2 1 0 0] -> [1 2 1]
        gx = gx[1:deg_gx+1]  # Discard the extra 0 in gx

        if gx == [0]
            println("Warning, cannot divide 0!\n")
            return (rem=[],quo=[])
        elseif deg_gx > deg_fx  # If the order of gx is higher than fx, the remainder is gx, the quotient is 0
            rx = fx
            qx = [0]
            return (rem=rx,quo=qx)
        else
            deg_diff = deg_fx - deg_gx  # degree difference between fx and gx
            qx = zeros(deg_diff+1)  # Initialize qx and rx
            rx = zeros(deg_gx+1)

            for i in 0:deg_diff
                # Add 0 to the gx vector to make gx and fx the same length
                # With the change of degree quotient, the position of gx is different each time
                zero_highdeg = zeros(Int64,i) 
                zero_lowdeg = zeros(Int64,deg_diff-i)
                gx_addzero = append!(zero_lowdeg,gx,zero_highdeg)

                # Calculate the quotient of each step
                # eg: Divide the highest order coefficient of fx by the highest order coefficient of gx for the first time
                qx[deg_diff+1-i] = fx[deg_fx+1-i]/gx[deg_gx+1]

                if !isinteger(qx[deg_diff+1-i])  # Fraction modulo if qx is not a integer
                    qx[deg_diff+1-i] = fractionmod(fx[deg_fx+1-i],gx[deg_gx+1],p)
                else  # integer mod
                    qx[deg_diff+1-i] = mod(qx[deg_diff+1-i],p) 
                end

                # Calculate the remainder, the remainder is the next divider
                fx = fx-gx_addzero.*qx[deg_diff+1-i]
            end

            rx = convert(Vector{Int64},mod.(fx,p))  # the newest fx is the remainder, do modulation
            qx = convert(Vector{Int64},mod.(qx,p))
        
            return (rx[1:getdeg(rx)+1],qx[1:getdeg(qx)+1])  # Discard extra 0s
        end
    else
        return "p is not a prime number. Please change the value of p." # p is not a prime
    end
end



"""
    vec_conv(a, b)

    Our own convolution function
    The multiplication of polynomials is actually the convolution
"""
function vec_conv(a, b) #a(x)*b(x) is actually the convolution of the two vector elements
    M = length(a)        
    N = length(b)
    c = zeros(Int64,M+N-1)
    for i = 1:M
        for j = 1:N
            c[i+j-1] += a[i]*b[j]  # find each element
        end
    end
    return c
end

vec_conv

In the above block, all the functions come from previous courseworks.

In the next block, two new functions are written.

In [69]:
"""
    find_E_and_L(Z,S,p,t)  

    Use Euclidian algorithm to find the remainder with the degree ≤ t-1 between fx and gx, and use Bezout's Identity to represent this remainder using fx and gx
"""
function find_E_and_L(Z,S,p,t)  
    if isprime(p) == false
        return  "p is not a prime number. Please change the value of p." 
    end
    
    (remainder,quotient) = polydiv(Z,S,p)  # Polynomial division, Find the remainder and quotient of Z / S in p-domain
    if remainder == [0] || getdeg(remainder) <= (t-1)  # if the degree of remainder ≤ t-1 (use remainder == [0] because the function 'getdeg' cannot process [0])
        return remainder, ones(Int64,1), -quotient
    else
        remainder, b_prev, a_prev = find_E_and_L(S,remainder,p,t)  # if not, iterate untill the degree of remainder ≤ t-1
        
        # Bezout's Identity
        b = mod.(a_prev,p)
        temp = mod.(vec_conv(a_prev,quotient),p)
        a = mod.(append!(b_prev,zeros(length(temp)-length(b_prev))) - temp,p)
        return remainder, b, a  # remainder = b*Z + a*S = a*S mod z^(d-1) where deg(remainder) ≤ t-1
    end
end

find_E_and_L

Function 'RSNum_KeyPolys'

In [70]:
using Polynomials


"""
    RSNum_KeyPolys(S,p,d,t)

    Find L and E use Euclidian algorithm to find the remainder with the degree ≤ t-1, and use Bezout's Identity to represent this remainder
"""
function RSNum_KeyPolys(S,p,d,t)
    t = floor(Int,(d-1)/2)  # maximum correctable errors
       
    # Z = z^(d-1)
    Z = zeros(Int64,d)
    Z[d] = 1
    
    E, ~, L = find_E_and_L(Z,S,p,t)  # Apply Z/s, find E and L
    println("8.  Locator polynomial L(z) = ",Polynomial(L,:z),"\n")  # show the results
    println("9.  Error evaluator polynomial E(z) = ",Polynomial(E,:z),"\n")
    return E, L
end

RSNum_KeyPolys

In [71]:
# Test
p = 17
n = 16
d = 7
t = floor(Int,(d-1)/2)
k = k_generator(p,d)
a = a_generator(p)
m,c = RSNum_Enc(p,k,a)
r = RSNum_Channel(p,d,c,t)
S = RSNum_Syndrome(p,n,k,a,r)
RSNum_KeyPolys(S,p,d,t)
nothing

1.  The chosen primitive element a = 5

2.  Message m is [0, 12, 1, 2, 11, 11, 16, 2, 13, 6]

3.  Codeword c is [6,

 12, 16, 3, 9, 9, 7, 14, 8, 5, 6, 13, 5, 1, 13, 9]

4.  Error position is [4, 6, 10]

5.  Error vector is [0, 0, 0, 3, 0, 7, 0, 0, 0, 14, 0, 0, 0, 0, 0, 0]

6.  Received word is [6, 12, 16, 6, 9, 16, 7, 14, 8, 2, 6, 13, 5, 1, 13, 9]

7.  Syndrome polynomial S(z) = 12 + 11*z + z^2 + 13*z^3 + 11*z^4 + 4*z^5



8.  Locator polynomial L(z) = 2 + 4*z + 2*z^2 + 7*z^3

9.  Error evaluator polynomial E(z) = 7 + 2*z + 2*z^2



## 7. Calculate estimated error $\hat{e}$ and estimated codeword $\hat{c}$
After getting $L(z)$ and $E(z)$, the error positions can be found from $L(z)$, and corresponding errors $e_i$ from $E(z)$.

Substitute $a^0,\alpha^{-1},\alpha^{-2},\cdots,\alpha^{-\left(p-2\right)}$ to $L(z)$, by this exhaustively computing, find all k that satisfy $L\left(\alpha^{-k}\right)=0$. The value of k represents error position.

As for the corresponding errors, $e_k=E\left(\alpha^{-k}\right)/\left(\alpha^k\mathrm{\Pi}_{j\neq k}\left(1-\alpha^j\alpha^{-k}\right)\right)=-E\left(\alpha^{-k}\right)/\frac{d}{dz}L\left(\alpha^{-k}\right)$. It is better to use the second one which has less complexity.

Then the codeword $\hat{c}$ can be recovered by $\hat{c}=r-\hat{e}$

In [72]:
using Polynomials


"""
    dfx(fx,p)

    Find the first derivative of fx
"""
function dfx(fx,p)
    dfx = zeros(Int64,length(fx)-1)  # Initialize the first derivative
    for i in 2:length(fx)
        dfx[i-1] = mod(fx[i]*(i-1),p)  # Calculate each element of the first derivative
    end
    return dfx
end



"""
    Substi(x,fx,p)

    Substitute x into f(x) in Fp. If use the function polynomial, it has the same problem as out abandoned method mentioned in "3.Generate message m and codeword c"
"""
function Substi(x,fx,p)
    L = length(fx)
    x_i = zeros(Int64,1,L)
    for i in 1:L
        x_i[i] = powermod(x,i-1,p)  # get corresponding values
    end
    result = mod.(x_i*fx,p)   # multiply each coefficient with its corresponding x^i
    return result[1]  # result is an 1-element vector, use result[1] to get its value
end



"""
    RSNum_Err(p,a,r,L,E)

    Estimate the error and recover the codeword
"""
function RSNum_Err(p,a,r,L,E)
    # Estimate the error position
    e_p = Int64[]  # Initialize the estimated error position
    for i in 1:p-1
        if mod(Substi(powermod(a,-(i-1),p),L,p),p) == 0  # substitute a^(-k) into L, check whether the result equals to 0
            push!(e_p,i)
        end
    end
    println("10. Estimated error position is ",e_p,"\n")

    # Estimate the error
    e_hat = zeros(Int64,p-1)  # Initialize the estimated error
    dL = dfx(L,p)  # the 1st derivative of L
    for i in 1:length(e_p)
        # e[k]        =                  - E[a^(-k)]                               /                 d[L(a^(-k))]/dz
        e_hat[e_p[i]] = fractionmod(mod(-1*Substi(powermod(a,-(e_p[i]-1),p),E,p),p), Substi(powermod(a,-(e_p[i]-1),p),dL,p), p)
    end
    println("11. Estimated error is ", e_hat,"\n")

    # Estimate the codeword
    c_hat = mod.(r' - e_hat, p)  # c = r - e
    println("12. Estimated codeword is ", vec(c_hat),"\n")
    return c_hat
end

RSNum_Err

In [73]:
# Test
p = 17
n = 16
d = 7
t = floor(Int,(d-1)/2)
k = k_generator(p,d)
a = a_generator(p)
m,c = RSNum_Enc(p,k,a)
r = RSNum_Channel(p,d,c,t)
S = RSNum_Syndrome(p,n,k,a,r)
E, L = RSNum_KeyPolys(S,p,d,t)
RSNum_Err(p,a,r,L,E)
nothing

1.  The chosen primitive element a = 5

2.  Message m is [10, 8, 3, 6, 4, 0, 9, 12, 7, 10]

3.  Codeword c is [1, 5, 16, 2, 9, 14, 11, 14, 14, 2, 7, 5, 9, 12, 1, 4]

4.  Error position is [6, 10, 11]

5.  Error vector is [0, 0, 0, 0, 0, 8

, 0, 0, 0, 14, 4, 0, 0, 0, 0, 0]

6.  Received word is [1, 5, 16, 2, 9, 5, 11, 14, 14, 16, 11, 5, 9, 12, 1, 4]

7.  Syndrome polynomial S(z) = 10 + 15*z + 15*z^2 + 10*z^3 + 11*z^5

8.  Locator polynomial L(z) = 6 + 11*z + 15*z^2 + 6*z^3

9.  Error evaluator polynomial E(z) = 9 + 13*z + 14*z^2



10. Estimated error position is [6, 10, 11]

11. Estimated error is [0, 0, 0, 0, 0, 8, 0, 0, 0, 14, 4, 0, 0, 0, 0, 0]

12. Estimated codeword is [1, 5, 16, 2, 9, 14, 11, 14, 14, 2, 7, 5, 9, 12, 1, 4]



## 8. Recover the message 
With the estimated codeword $\hat{c}$, the estimated message $\hat{m}$ can be recovered. Since the original message $m$ is converted to $c$ by $m*G=c$, it can be converted back by $m=cG^{-1}$, hence $\hat{m}=\hat{c}G^{-1}$

Obviously, we can directly take the inverse of $G$, which is $inv(G)$, and get the result. However, elements in the result of $inv(G)$ are float point numbers, which cannot be applied modulation. Therefore, we designed a function 'myinv', which is our own matrix inverse function to apply modulation to matrix elements during calculation.

But, this function also has a problem, which will be explained later.

In [74]:
## Abandoned method, explained bellow this block. Just for demonstration
## !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

using LinearAlgebra


"""
    inv_with_mod(Mat,p)

    Our own function for matrix inverse
"""
function inv_with_mod(Mat,p)
    L = LinearAlgebra.checksquare(Mat)  # Check whether matrix A is a square matrix, if true, return its size
    Co = zeros(Int64,L,L)  # Cofactor 
    
    for i = 1:L
        # Use the elimination matrix to eliminate  the row or column of A to get the minor
        El = fill(true,L)  # El has L elements with type true
        El[i] = false  # Set the ith element to be false, which can eliminate the ith row or column
        Mi1 = Mat[:,El]  # Mi1 is used to find the minor, here, the ith column of A is eliminated

        for n = 1:L
            El = fill(true,L)
            El[n] = false  # Set the nth element to be false, which can eliminate the nth row or column
            Mi = Mi1[El,:]  # Mi is the minor, here, the nth column of Mi1 is eliminated, which means that Mi is the matrix that eliminate the ith column and nth row of A

            # Find each element of the Cofactor matrix
            Co[n,i] = mod((-1)^(n+i)*round(det(Mi)),p)  # Use round(det(Mi)) because det() return float point numbers, e.g. 3.0000000000000001
        end
    end
    return mod.(fractionmod(1,round(det(Mat)),p)*Co, p)  # Multiply 1/det(A) with the Cofactor matrix, the inverse of A is found
end

inv_with_mod

In [75]:
# Test
Mat = [1 5 25 3 15 14 9 45 42 27 13 4 20 39 12 60 56 36 58 46 47 52 16 19 34 48 57 41 22 49 1;  # Mat is a 31×31 Matrix
     1 8 3 24 9 11 27 33 20 38 60 53 58 37 52 50 34 28 41 23 1 8 3 24 9 11 27 33 20 38 60; 
     1 25 15 9 42 13 20 12 56 58 47 16 34 57 22 1 25 15 9 42 13 20 12 56 58 47 16 34 57 22 1; 
     1 3 9 27 20 60 58 52 34 41 1 3 9 27 20 60 58 52 34 41 1 3 9 27 20 60 58 52 34 41 1; 
     1 15 42 20 56 47 34 22 25 9 13 12 58 16 57 1 15 42 20 56 47 34 22 25 9 13 12 58 16 57 1; 
     1 24 27 38 58 50 41 8 9 33 60 37 34 23 3 11 20 53 52 28 1 24 27 38 58 50 41 8 9 33 60; 
     1 59 4 53 16 29 3 55 12 37 48 26 9 43 36 50 22 17 27 7 47 28 5 51 20 21 19 23 15 31 60; 
     1 9 20 58 34 1 9 20 58 34 1 9 20 58 34 1 9 20 58 34 1 9 20 58 34 1 9 20 58 34 1; 
     1 11 60 50 1 11 60 50 1 11 60 50 1 11 60 50 1 11 60 50 1 11 60 50 1 11 60 50 1 11 60; 
     1 42 56 34 25 13 58 57 15 20 47 22 9 12 16 1 42 56 34 25 13 58 57 15 20 47 22 9 12 16 1; 
     1 33 52 8 20 50 3 38 34 24 60 28 9 53 41 11 58 23 27 37 1 33 52 8 20 50 3 38 34 24 60; 
     1 4 16 3 12 48 9 36 22 27 47 5 20 19 15 60 57 45 58 49 13 52 25 39 34 14 56 41 42 46 1; 
     1 43 19 24 56 29 27 2 25 38 48 51 58 54 4 50 15 35 41 55 47 8 39 30 9 21 49 33 16 17 60; 
     1 32 48 11 47 40 60 29 13 50 14 21 1 32 48 11 47 40 60 29 13 50 14 21 1 32 48 11 47 40 60; 
     1 12 22 20 57 13 34 42 16 9 47 15 58 25 56 1 12 22 20 57 13 34 42 16 9 47 15 58 25 56 1; 
     1 53 3 37 9 50 27 28 20 23 60 8 58 24 52 11 34 33 41 38 1 53 3 37 9 50 27 28 20 23 60; 
     1 21 14 50 13 29 60 40 47 11 48 32 1 21 14 50 13 29 60 40 47 11 48 32 1 21 14 50 13 29 60; 
     1 58 9 34 20 1 58 9 34 20 1 58 9 34 20 1 58 9 34 20 1 58 9 34 20 1 58 9 34 20 1; 
     1 46 42 41 56 14 34 39 25 52 13 49 58 45 57 60 15 19 20 5 47 27 22 36 9 48 12 3 16 4 1; 
     1 37 27 23 58 11 41 53 9 28 60 24 34 38 3 50 20 8 52 33 1 37 27 23 58 11 41 53 9 28 60; 
     1 47 13 1 47 13 1 47 13 1 47 13 1 47 13 1 47 13 1 47 13 1 47 13 1 47 13 1 47 13 1; 
     1 2 4 8 16 32 3 6 12 24 48 35 9 18 36 11 22 44 27 54 47 33 5 10 20 40 19 38 15 30 60; 
     1 10 39 24 57 21 27 26 16 38 14 18 58 31 5 50 12 59 41 44 13 8 19 7 9 29 46 33 25 6 60; 
     1 16 12 9 22 47 20 15 57 58 13 25 34 56 42 1 16 12 9 22 47 20 15 57 58 13 25 34 56 42 1; 
     1 19 56 27 25 48 58 4 15 41 47 39 9 49 16 60 42 5 34 36 13 3 57 46 20 14 22 52 12 45 1; 
     1 30 46 38 42 40 41 10 56 33 14 54 34 44 39 11 25 18 52 35 13 24 49 6 58 32 45 8 57 2 60; 
     1 48 47 60 13 14 1 48 47 60 13 14 1 48 47 60 13 14 1 48 47 60 13 14 1 48 47 60 13 14 1; 
     1 28 52 53 20 11 3 23 34 37 60 33 9 8 41 50 58 38 27 24 1 28 52 53 20 11 3 23 34 37 60; 
     1 18 19 37 56 32 27 59 25 23 48 10 58 7 4 11 15 26 41 6 47 53 39 31 9 40 49 28 16 44 60; 
     1 49 22 41 57 48 34 19 16 52 47 46 58 36 56 60 12 39 20 4 13 27 42 45 9 14 15 3 25 5 1; 
     1 54 49 23 22 29 41 18 57 28 48 30 34 6 19 50 16 10 52 2 47 37 46 44 58 21 36 53 56 35 60]
p = 61
inv_with_mod(Mat,p)

31×31 Matrix{Int64}:
 43  30  24  20   9  60  17  22  29  …  19  43   8  45   2  14  39   7   4
  3  33  32  49  22  13  53   3  39     46   8  26  36  52  51  49   5  35
 27  27  40  18  16  56  56  47   3     13  32  41  24  11  55  40  48  47
 14  27  38  53  42  55  17  54   8     34  58  36  44  43  13  36   5  36
 25  57  45  56   8  40   6  35  28     51  18  20  15  22   7   9   6  27
  7  18  42  29  12  29  21  15  28  …  28  27  46  16  47  11   0  44  10
 11  34  21   3  47  23  45  55  50     20  39  21  25  54  23  38   8  40
  8  41  28  31  47  26   0  60   1     18  39  10  32  27  12  15  42  31
 52  60  15   3  29  22  35  23  42     52   3  59   7  52  59  58  45   2
 48   7   0   7  54  10  20  59  35     18   4   7   3   3  13  44  21  39
  ⋮                   ⋮              ⋱               ⋮                   ⋮
 21  43  17  33  36  56  40  48  19      0  11  21  27   9  28  48  47  37
 56   4  26  49  17  50  17  55   0     19  37  60  37  54  38  38  35  17
 40 

However, the above method is abandoned. The reason is that the result of det(A) is not precise as well. When the result of det(A) is very large, e.g. the Matrix A used in #Test in the above block, the determinant of A is -3.449251804558317e54, which is not precise.

Therefore, we decide to use Gaussian Elimination to calculate the estimated m, $\hat{m}G=\hat{c}$. Since the Gaussian Elimination should be in the form of $Ax = b$, transpose is taken, $G^{T}\hat{m}^{T}=\hat{c}^{T}$.

In our function 'Gaussian_Elimination', for $Ax = b$, the steps can be described as,
1. Eliminate the 1st element of the 2nd row, the 3rd row, ..., the last row, by subtracting the 1st row * an appropriate factor. Then, eliminate the 2nd element of the 3rd row, the 4th row, ..., the last row, by subtracting the 2nd row * an appropriate factor. Repeat these steps, until the matrix A becomes an upper-triangular matrix. The same linear transformation should also be applied to b.
2. In the last row, we can get $A[L,L]*x[L]=b[L]$, hence $x[L] = b[L]/A[L,L]$. Substitute $x[L]$ to the second row from the bottom, $x[L-1]$ can be found. Then, substitute $x[L]$ and $x[L-1]$ to the third row from the bottom, $x[L-2]$ can be found. Repeat the substitution, each element of x can be calculated.

In [76]:
"""
    Gaussian_Elimination(A,b,p)

    Function for Gaussian Elimination
"""
function Gaussian_Elimination(A,b,p)
    L = LinearAlgebra.checksquare(A)  # Check whether matrix A is a square matrix, if true, return its size
    x = zeros(Int64,L,1)  # Initialize x, Ax = b, target is to solve x
    
    # Gaussian Elimination, transfer A to an upper-triangular matrix
    for i in 1:L-1
       for j in i+1:L
          a = fractionmod(A[j,i],A[i,i],p)  # a = A[j,i]/A[i,i]
          for k in i+1:L
             A[j,k] = mod(A[j,k] - A[i,k] * a, p)  # Multiply the ith row with a, use jth row to subtract it
          end
          A[j,i] = 0  # After multiplying the ith row with a, the ith element of the jth row becomes the same as the ith element of the ith row, hence the result of subtraction is 0
          
          b[j] = mod(b[j] - b[i] * a,p)    #  Apply the same thing to b
       end
    end
    
    # Substitution, now we have an upper-triangular matrix A times x equals to b (A and b are different from the original A and b)
    x[L] = mod(fractionmod(b[L],A[L,L],p),p)  # Find x[L]
    
    # Substitute x[L], x[L-1],...,x[i+1] to calculate x[i]
    for i in L-1:-1:1
       sum = 0;
       for j in i+1:L
          sum = mod(sum + A[i,j] * x[j],p)   
       end
       x[i] = mod(fractionmod(b[i]-sum,A[i,i],p),p)
    end
    return x
end
 
 
 
"""
    RSNum_Message(p,k,a,c)
 
    Function to recover message m from the codeword c
"""
function RSNum_Message(p,k,a,c)
    # Initialize the generator matrix
    G = zeros(Int64,k,p-1)  
    for i in 1:k
        for j in 1:p-1
            G[i,j] = powermod(a,(i-1)*(j-1),p)
        end
    end

    # Randomly choose k columns of the generator matrix G, make it become a square matrix
    index = sort(sample(collect(1:p-1),k,replace=false),by=abs)

    # take the transpose since Gaussian Elimination should be in the form of Ax = b
    G_T = (G[:,index])'

    # Choose the corresponding elements of c_hat
    c_hat = c[index]

    # Use Gaussian Elimination to find the result, we have the function G_T * m_hat = c_hat (m and c are vectors, hence no need to take their transpose in code)
    m_hat = vec(Gaussian_Elimination(G_T,c_hat,p))
    println("13. Estimated message is ", m_hat,"\n")
    return m_hat
end

RSNum_Message

In [77]:
# Test
p = 17
n = 16
d = 7
t = floor(Int,(d-1)/2)
k = k_generator(p,d)
a = a_generator(p)
m,c = RSNum_Enc(p,k,a)
r = RSNum_Channel(p,d,c,t)
S = RSNum_Syndrome(p,n,k,a,r)
E, L = RSNum_KeyPolys(S,p,d,t)
c_hat = RSNum_Err(p,a,r,L,E)
RSNum_Message(p,k,a,c_hat)
nothing

1.  The chosen primitive element a = 12

2.  Message m is [15, 1, 14, 12, 2, 3, 13, 4, 2, 10]

3.  Codeword c is [8, 0, 6, 3, 0, 2, 14, 11, 16, 14, 16, 14, 1, 4, 7, 5]

4.  Error position is 

[5, 7, 9]

5.  Error vector is [0, 0, 0, 0, 12, 0, 14, 0, 15, 0, 0, 0, 0, 0, 0, 0]

6.  Received word is [8, 0, 6, 3, 12, 2, 11, 11, 14, 14, 16, 14, 1, 4, 7, 5]

7.  Syndrome polynomial S(z) = 16 + 8*z + 9*z^2 + 13*z^3 + 11*z^4 + 15*z^5

8.  Locator polynomial L(z) = 2 + 6

*z + 5*z^2 + z^3

9.  Error evaluator polynomial E(z) = 15 + 10*z + 10*z^2

10. Estimated error position is [5, 7, 9]

11. Estimated error is [0, 0, 0, 0, 12, 0, 14, 0, 15, 0, 0, 0, 0, 0, 0, 0]

12. Estimated codeword is [8, 0, 6, 3, 0, 2, 14, 11, 16, 14, 16, 14, 1, 4, 7, 5]



13. Estimated message is [15, 1, 14, 12, 2, 3, 13, 4, 2, 10]



## 9.  Decoding process
Implement the procedures above to complete the whole decoding process.

In [78]:
"""
    RSNum_Dec(p,n,k,d,a,r)

    Decoding function, combine steps 5-8
"""
function RSNum_Dec(p,n,k,d,a,r)
    S = RSNum_Syndrome(p,n,k,a,r)
    E, L = RSNum_KeyPolys(S,p,d,t)
    c_hat = RSNum_Err(p,a,r,L,E)
    m_hat = RSNum_Message(p,k,a,c_hat)
    return m_hat
end

RSNum_Dec

In [79]:
# Test
p = 17
n = 16
d = 7
t = floor(Int,(d-1)/2)
k = k_generator(p,d)
a = a_generator(p)
m,c = RSNum_Enc(p,k,a)
r = RSNum_Channel(p,d,c,t)
RSNum_Dec(p,n,k,d,a,r)
nothing

1.  The chosen primitive element a = 14

2.  Message m is [13, 6, 7, 9, 6, 8, 14, 5, 11, 7]

3.  Codeword c is [1, 1, 15, 0, 15, 4, 16, 6, 16, 8,

 9, 4, 3, 1, 15, 9]

4.  Error position is [7, 13, 16]

5.  Error vector is [0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 8, 0, 0, 4]

6.  Received word is [1, 1, 15, 0, 15, 4, 9, 6, 16, 8, 9, 4, 11, 1, 15, 13]



7.  Syndrome polynomial S(z) = 5 + 6*z + 10*z^2 + 14*z^3 + 7*z^4 + z^5

8.  Locator polynomial L(z) = 16 + 13*z + 3*z^2 + 14*z^3

9.  Error evaluator polynomial E(z) = 12 + 8*z + 15*z^2

10. Estimated error position is [7, 13, 16]

11. Estimated error is [0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 8, 0, 0, 4]

12. Estimated codeword is [1, 1, 

15, 0, 15, 4, 16, 6, 16, 8, 9, 4, 3, 1, 15, 9]

13. Estimated message is [13, 6, 7, 9, 6, 8, 14, 5, 11, 7]



## 10.  Final Function
The final combined function.

In [80]:
"""
    RSNum(p,n,k,d)

    Final function
"""
function RSNum(p,n,k,d,t)
    if isprime(p) == false
        println("p is not a prime number. Please change the value of p.\n")
        return "Warning"
    end

    # Randomly choose a primitive element from the set
    a = a_generator(p)

    # Generate the message and codeword
    m, c = RSNum_Enc(p,k,a)

    # Generate noise and add this noise to the codeword
    r = RSNum_Channel(p,d,c,t)

    # Recover the message
    m_hat = RSNum_Dec(p,n,k,d,a,r)

    # Check the correctness
    if m == m_hat
        println("🙂🙂🙂 Recovery Successful 🙂🙂🙂\n")
        flag = true
    else
        println("!!!!! Fail to recover message m !!!!!\n")
        flag = false
    end

    println("Put original message and estimated message together to compare:")
    println(m)
    println(m_hat)
    return flag
end

RSNum

In [81]:
# Test
p = 17
n = p-1
d = 7
t = floor(Int,(d-1)/2)
k = n-d+1
RSNum(p,n,k,d,t)
nothing

1.  The chosen primitive element a = 10

2.  Message m is [11, 13, 15, 12, 13, 16, 8, 10, 10, 16]

3.  Codeword c is [5, 5, 11, 5, 1, 15, 8, 7, 7, 15, 10, 12, 4, 7, 3, 10]

4.  Error position is [1, 10

, 12]

5.  Error vector is [4, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 3, 0, 0, 0, 0]

6.  Received word is [9, 5, 11, 5, 1, 15, 8, 7, 7, 9, 10, 15, 4, 7, 3, 10]

7.  Syndrome polynomial S(z) = 5 + 9*z + 16*z^2 + 2*z^3 + 4*z^4 + 12*z^5

8.  Locator polynomial L(z) = 16 + 11*z + 3*z^2 + 4*z^3

9.  Error evaluator polynomial E(z) = 12 + 12*z + 13*z^2

10. Estimated error position is [1, 10, 12]

11. Estimated error is [4, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 3, 0, 0, 0, 0]

12. Estimated codeword is [5, 5, 11, 5

, 1, 15, 8, 7, 7, 15, 10, 12, 4, 7, 3, 10]

13. Estimated message is [11, 13, 15, 12, 13, 16, 8, 10, 10, 16]

🙂🙂🙂 Recovery Successful 🙂🙂🙂

Put original message and estimated message together to compare:
[11, 13, 15, 12, 13, 16, 8, 10, 10, 16]
[11, 13, 15, 12, 13, 16, 8, 10, 10, 16]


### Explanation for the result above

The whole process of RS code is:

1. Determine basic parameters $p$, $n$, $k$, $d$, where $n=p-1, d=n-k+1$
2. Randomly choose a primitive element in $\mathbb{F}_{p}$, which is shown in the above result (1)
3. Generate message randomly, shown in result (2)
4. Based on this message, find code codeword $c$, shown in result (3)
5. Then, this codeword is transfer through the channel, and random noise is added to the codeword. The position of errors is in result (4), the error vector is in result (5). Also, the received word is in result (6).
6. Based on the received word, syndrome polynomial is shown in result (7), calculated by $s=y*H'$, where H is the parity-check matrix.
7. Then, use the syndrome polynomial to find locator polynomial $L(z)$ and error evaluator polynomial $E(z)$, shown in result (8) and (9), respectively.
8. Use $L(z)$ and $E(z)$ to estimate the error positions and error vector, shown in result (10) and (11), respectively.
9. Use $c = r - e$ to estimate the codeword, shown in result (11).
10. Finally, use Gaussian Elimination to recover the message from the codeword using the equation $m * G = c$ or in the standard form $G^T * m^T = c^T$.

Obviously, the estimated vectors are precisely the same as the original ones. The result demonstrates that our implementation is correct.

### 3.1.2 RS Codes on $\mathbb{F}_{p}$

The above code should work well for other choices of $p$, $\alpha$, $[n,k,d]$. Try another $p$ with $30<p<100$ and demonstrate the correctness of your code. 

This is the same as 3.1.1, just change the value of p and d.

In [82]:
# Test
p = 97
d = 30
t = floor(Int,(d-1)/2)
n = p-1
k = n-d+1
RSNum(p,n,k,d,t)
nothing

1.  The chosen primitive element a = 21

2.  Message m is [34, 90, 72, 87, 84, 3

, 53, 2, 86, 28, 15, 86, 67, 32, 26, 55, 78, 89, 78, 40, 3, 79, 31, 16, 30, 59, 34, 96, 42, 19, 23, 74, 67, 63, 24, 67, 91, 8, 85, 26, 16, 30, 69, 10, 25, 43, 25, 24, 23, 17, 32, 87, 6, 63, 65, 49, 92, 23, 13, 89, 90, 59, 42, 75, 21, 48, 28]

3.  Codeword c is [5, 75, 63, 18, 1, 20, 55, 84, 19, 54, 65, 24, 53, 36, 90, 5, 8, 78, 57, 31, 1, 89, 39, 7, 93, 86, 69, 40, 53, 72, 61, 38, 65, 40, 54, 81, 30, 55, 23, 35, 72, 33, 53, 83, 48, 79, 81, 15, 31, 67, 38, 85, 45, 34, 68, 96, 3, 42, 54, 65, 6, 77, 35, 61, 86, 61, 62, 28, 28, 82, 58, 67, 90, 93, 4, 60, 64, 21, 75, 37, 25, 88, 14, 25, 67, 81, 6, 30, 31, 81, 46, 89, 0, 13, 60, 4]

4.  Error position is 

[5, 6, 12, 13, 29, 45, 47, 53, 62, 64, 70, 72, 94, 95]

5.  Error vector is [0, 0, 0, 0, 73, 27, 0, 0, 0, 0, 0, 96, 35, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 0, 17, 0, 0, 0, 0, 0, 30, 0, 0, 0, 0, 0, 0, 0, 0, 68, 0, 50, 0, 0, 0, 0, 0, 32,

 0, 45, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 3, 0]

6.  Received word is [5, 75, 63, 18, 74, 47, 55, 84, 19, 54, 65, 23, 88, 36, 90, 5, 8, 78, 57, 31, 1, 89, 39, 7, 93, 86, 69, 40, 84, 72, 61, 38, 65, 40, 54, 81, 30, 55, 23, 35, 72, 33, 53, 83, 18, 79, 1, 15, 31, 67, 38, 85, 75, 34, 68, 96, 3, 42, 54, 65, 6, 48, 35, 14, 86, 61, 62, 28, 28, 17, 58, 15, 90, 93, 4, 60, 64, 21, 75, 37, 25, 88, 14, 25, 67, 81, 6, 30, 31, 81, 46, 89, 0, 26, 63, 4]

7.  Syndrome polynomial S(z) = 59 + 61*z + 36*z^2 + 25*z^3 + 68*z^4 + 78*z^5 + 90*z^6 + 51*z^7 + 9*z^8 + 66*z^9 + 25*z^10 + 86*z^11 + 73*z^12 + 69*z^13 + 15*z^14 + 87*z^15 + 52*z^16 + 70*z^17 + 3*z^18 + 18*z^19 + 20*z^20 + 96*z^21 + 76*z^22 + 64*z^23 + 19*z^24 + 68*z^25 + 64*z^26 + 53*z^27 + 56*z^28

8.  Locator polynomial L(z) = 96 + 59*z + 52*z^3 + 38*z^4 + 44*z^5 + 13*z^6 + 84*z^7 + 15*z^8 + 9*z^9 + 66*z^10 + 39*z^11 + 35*z^12 + 47*z^13 + 92*z^14

9.  Error evaluator polynomial E(z) = 38 + 25*z + 71*z^2 + 26*z^3 + 

, 6, 12, 13, 29, 45, 47, 53, 62, 64, 70, 72, 94, 95]

11. Estimated error is [0, 0, 0, 0, 73, 27, 0, 0, 0, 0, 0, 96, 35, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 0, 17, 0, 0, 0, 0, 0, 30, 0, 0, 0, 0, 0, 0, 0, 0, 68, 

0, 50, 0, 0, 0, 0, 0, 32, 0, 45, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 3, 0]

12. Estimated codeword is [5, 75, 63, 18, 1, 20, 55, 84, 19, 54, 65, 24, 53, 36, 90, 5, 8, 78, 57, 31, 1, 89, 39, 7, 93, 86, 69, 40, 53, 72, 61, 38, 65, 40, 54, 81, 30, 55, 23, 35, 72, 33, 53, 83, 48, 79, 81, 15, 31, 67, 38, 85, 45, 34, 68, 96, 3, 42, 54, 65, 6, 77, 35, 61, 86, 61, 62, 28, 28, 82, 58, 67, 90, 93, 4, 60, 64, 21, 75, 37, 25, 88, 14, 25, 67, 81, 6, 30, 31, 81, 46, 89, 0, 13, 60, 4]

13. Estimated message is [34, 90, 72, 87, 84, 3,

 53, 2, 86, 28, 15, 86, 67, 32, 26, 55, 78, 89, 78, 40, 3, 79, 31, 16, 30, 59, 34, 96, 42, 19, 23, 74, 67, 63, 24, 67, 91, 8, 85, 26, 16, 30, 69, 10, 25, 43, 25, 24, 23, 17, 32, 87, 6, 63, 65, 49, 92, 23, 13, 89, 90, 59, 42, 75, 21, 48, 28]

🙂🙂🙂 Recovery Successful 🙂🙂🙂

Put original message and estimated message together to compare:
[34, 90, 72, 87, 84, 3, 53, 2, 86, 28, 15, 86, 67, 32, 26, 55, 78, 89, 78, 40, 3, 79, 31, 16, 30, 59, 34, 96, 42, 19, 23, 74, 67, 63, 24, 67, 91, 8, 85, 26, 16, 30, 69, 10, 25, 43, 25, 24, 23, 17, 32, 87, 6, 63, 65, 49, 92, 23, 13, 89, 90, 59, 42, 75, 21, 48, 28]
[34, 90, 72, 87, 84, 3, 53, 2, 86, 28, 15, 86, 67, 32, 26, 55, 78, 89, 78, 40, 3, 79, 31, 16, 30, 59, 34, 96, 42, 19, 23, 74, 67, 63, 24, 67, 91, 8, 85, 26, 

16, 30, 69, 10, 25, 43, 25, 24, 23, 17, 32, 87, 6, 63, 65, 49, 92, 23, 13, 89, 90, 59, 42, 75, 21, 48, 28]


In [83]:
# Test
# This result should be fail !!!!!!!!!!!!!!

p = 97
d = 30
t = floor(Int,(d-1)/2) + 1  # If t exceed its limitation 
n = p-1
k = n-d+1
RSNum(p,n,k,d,t)
nothing

1.  The chosen primitive element a = 29

2.  Message m is [56, 96, 5, 30, 59, 46, 67, 64, 35, 45, 51, 49, 61, 70, 85, 57, 5, 12, 33, 26, 61, 2, 48, 51, 74, 76, 50, 77, 31, 89, 64, 17, 20, 77, 54, 55

, 42, 34, 95, 40, 30, 30, 13, 31, 69, 0, 79, 85, 42, 64, 4, 50, 21, 27, 89, 91, 59, 8, 51, 83, 51, 29, 86, 56, 24, 5, 52]

3.  Codeword c is [37, 4, 91, 95, 53, 50, 54, 34, 58, 62, 21, 77, 81, 85, 52, 5, 24, 91, 67, 61, 50, 51, 59, 66, 59, 10, 82, 79, 50, 5, 28, 8, 78, 3, 40, 58, 31, 45, 14, 35, 20, 42, 40, 26, 54, 24, 50, 8, 94, 67, 95, 45, 17, 72, 13, 67, 14, 11, 77, 57, 5, 83, 69, 7, 39, 49, 71, 26, 82, 30, 54, 46, 54, 15, 96, 12, 59, 58, 59, 94, 3, 23, 10, 17, 71, 26, 64, 12, 37, 27, 36, 23, 51, 63, 13, 76]

4.  Error position is [4, 11, 15, 29, 31, 35, 45, 51, 58, 65, 74, 75, 79, 90, 91]

5.  Error vector is [0, 0, 0, 44, 0, 0, 0, 0, 0, 0, 21, 0, 0, 0, 77, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0, 11, 0, 0, 0, 73, 0, 0, 0, 0, 0, 0, 0, 0, 0, 52, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 82, 0, 0, 0, 0, 0, 0, 93, 0, 0, 0, 0, 0, 0, 0, 0, 93, 54, 0, 0, 0, 52, 0, 0, 0, 0

, 0, 0, 0, 0, 0, 0, 21, 80, 0, 0, 0, 0, 0]

6.  Received word is [37, 4, 91, 42, 53, 50, 54, 34, 58, 62, 42, 77, 81, 85, 32, 5, 24, 91, 67, 61, 50, 51, 59, 66, 59, 10, 82, 79, 63, 5, 39, 8, 78, 3, 16, 58, 31, 45, 14, 35, 20, 42, 40, 26, 9, 24, 50, 8, 94, 67, 96, 45, 17, 72, 13, 67, 14, 93, 77, 57, 5, 83, 69, 7, 35, 49, 71, 26, 82, 30, 54, 46, 54, 11, 53, 12, 59, 58, 14, 94, 3, 23, 10, 17, 71, 26, 64, 12, 37, 48, 19, 23, 51

, 63, 13, 76]

7.  Syndrome polynomial S(z) = 10 + 82*z + 61*z^2 + 43*z^3 + 60*z^4 + 87*z^5 + 48*z^6 + 14*z^7 + 24*z^8 + 16*z^9 + 46*z^11 + 33*z^12 + 85*z^13 + 61*z^14 + 27*z^15 + 43*z^16 + 40*z^17 + 21*z^18 + 91*z^19 + 31*z^20 + 21*z^21 + 55*z^22 + 29*z^23 + 11*z^24 + 89*z^25 + 65*z^26 + 63*z^28

8.  Locator polynomial L(z) = 61 + 94*z + 55*z^2 + 60*z^3 + 48*z^4 + 69*z^5 + 19*z^6 + 42*z^7 + 75*z^8 + 82*z^9 + 20*z^10 + 70*z^11 + 72*z^12 + 92*z^13 + 47*z^14 + 86*z^15

9.  Error evaluator polynomial E(z) = 28 + 25*z + 48*z^2 + 81*z^3 + 64*z^4 + 64*z^5 + 57*z^6 + 80*z^7 + 15*z^8 + 36*z^9 + 32*z^10 + 85*z^11 + 5*z^12 + 25*z^13

10. Estimated error position is [6, 87]

11. Estimated error is [0, 0, 0, 0, 0, 39, 0, 0, 0, 0, 0, 0, 0

, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 86, 0, 0, 0, 0, 0, 0, 0, 0, 0]

12. Estimated codeword is [37, 4, 91, 42, 53, 11, 54, 34, 58, 62, 42, 77, 81, 85, 32, 5, 24, 91, 67, 61, 50, 51, 59, 66, 59, 10, 82, 79, 63, 5, 39, 8, 78, 3, 16, 58, 31, 45, 14, 35, 20, 42, 40, 

26, 9, 24, 50, 8, 94, 67, 96, 45, 17, 72, 13, 67, 14, 93, 77, 57, 5, 83, 69, 7, 35, 49, 71, 26, 82, 30, 54, 46, 54, 11, 53, 12, 59, 58, 14, 94, 3, 23, 10, 17, 71, 26, 75, 12, 37, 48, 19, 23, 51, 63, 13, 76]

13. Estimated message is [24, 48, 21, 94, 18, 91, 90, 3, 84, 27, 39, 44, 73, 64, 5, 75, 41, 52, 76, 94, 48, 62, 95, 74, 64, 56, 78, 12, 55, 23, 2, 66, 10, 60, 61, 9, 15, 0, 94, 85, 86, 77, 20, 45, 24, 4, 39, 82, 76, 88, 22, 89, 41, 49, 55, 3, 6, 91, 68, 8, 18, 95, 33, 67, 45, 5, 19]

!!!!! Fail to recover message m !!!!!

Put original message and estimated message together to compare:
[56, 96, 5, 30, 59, 46, 67, 64, 35, 45, 51, 49, 61, 70, 85, 57, 5, 12, 33, 26, 61, 2, 48, 51, 74, 76, 50, 77, 31, 89, 64, 17, 20, 77, 54, 55, 42, 34, 95, 40, 30, 30, 13, 31, 69, 0, 79, 85, 42, 64, 4, 50, 21, 27, 89, 91, 59, 8, 51

, 83, 51, 29, 86, 56, 24, 5, 52]
[24, 48, 21, 94, 18, 91, 90, 3, 84, 27, 39, 44, 73, 64, 5, 75, 41, 52, 76, 94, 48, 62, 95, 74, 64, 56, 78, 12, 55, 23, 2, 66, 10, 60, 61, 9, 15, 0, 94, 85, 86, 77, 20, 45, 24, 4, 39, 82, 76, 88, 22, 89, 41, 49, 55, 3, 6, 91, 68, 8, 18, 95, 33, 67, 45, 5, 19]


## 3.2 RS Codes on $\mathbb{F}_{p^k}$ (50%)

### 3.2.1 RS Codes on $\mathbb{F}_{16}$

Now we consider the finite field $\mathbb{F}_{16} = \mathbb{F}_2[z]/z^4 + z +1$, and an RS code $\mathcal{C} \subset \mathbb{F}_{16}$ with parameters $[15,k,7]$. 

Repeat the steps in 3.1.1 and change the function names from `RSNum_...` to `RSPoly_...`. Provide necessary documentation. 

## 1.  Declare our basis in $\mathbb{F}_{p^k}$

In [84]:
using StatsBase


"""
    Table_generator(q,p,f_x)

    Relate the corresponding a^k with a polynomial, and the row number of Table represent the index of corresponding a^k

    e.g. F2^4
    0   0     [0 0 0 0]       8   a^7   [1 1 0 1]
    1   a^0   [1 0 0 0]       9   a^8   [1 0 1 0]
    2   a^1   [0 1 0 0]      10   a^9   [0 1 0 1]
    3   a^2   [0 0 1 0]      11   a^10  [1 1 1 0]
    4   a^3   [0 0 0 1]      12   a^11  [0 1 1 1]
    5   a^4   [1 1 0 0]      13   a^12  [1 1 1 1]
    6   a^5   [0 1 1 0]      14   a^13  [1 0 1 1]
    7   a^6   [0 0 1 1]      15   a^14  [1 0 0 1]
"""
function  Table_generator(q,p,f_x)
    L = getdeg(f_x)  # The length of the Table
    Table = []
    for i = 0:q-1
        if i == 0  # The first row is for zero
            pushfirst!(Table,zeros(Int64,L))
        else  # Build the table based on 'polydiv', 1 / f_x
            pol,~ = polydiv(prepend!([1],zeros(Int64,i-1)),f_x,p)
            append!(pol, zeros(Int64,(L-length(pol))))  # Zero Padding
            push!(Table,pol)
        end
    end
    return Table
end

Table_generator

In [85]:
# Test
q = 16
p = 2
f_x = [1,1,0,0,1]  # Fq/f(x)
Table_generator(q,p,f_x)

16-element Vector{Any}:
 [0, 0, 0, 0]
 [1, 0, 0, 0]
 [0, 1, 0, 0]
 [0, 0, 1, 0]
 [0, 0, 0, 1]
 [1, 1, 0, 0]
 [0, 1, 1, 0]
 [0, 0, 1, 1]
 [1, 1, 0, 1]
 [1, 0, 1, 0]
 [0, 1, 0, 1]
 [1, 1, 1, 0]
 [0, 1, 1, 1]
 [1, 1, 1, 1]
 [1, 0, 1, 1]
 [1, 0, 0, 1]

In [86]:
# The bellow two functions are based on 'Table_generator', change the index to polynomials in Table or change back.


"""
    Poly_to_Index(x,Table)

    Based on 'Table_generator', convert the polynomial in the Table to corresponding Index
"""
function Poly_to_Index(x,Table)
    if x isa Vector{Any}
        Index = zeros(Int64,length(x))
        for j in 1:length(x)
            for i in 1: length(Table)
                if x[j] == Table[i]  # Check which row equals to x, output the Index
                    Index[j] = i-1
                end
            end
        end
    elseif x isa Vector{Int}
        for i in 1: length(Table)
            if x == Table[i]  # Check which row equals to x, output the Index
                Index = i-1
            end
        end
    end
    return Index
end



"""
    Index_to_Poly(Index,Table)

    Based on 'Table_generator', convert the Index in the Table to corresponding polynomial
"""
function Index_to_Poly(Index,Table)
    if Index isa Int64
        return Table[Mod_Poly(Index,length(Table))+1]  # Output the row based on the Index
    else   # if x_dec is a vector
        x = []
        for i = 1:length(Index)
            push!(x,Table[Mod_Poly(Index[i],length(Table))+1])  # Output the row based on the Index
        end
    end
    return x
end

Index_to_Poly

In [87]:
# Basic midulation function


"""
    Mod_Poly(a,q) 

    Modulation in the field F2^k

    It produce a number ring with period q - 1 (except 0)
    e.g. F2^4

    -19 -18 -17 -16 -15 -14 -13 -12 -11 -10  -9  -8  -7  -6  -5  -4  -3  -2  -1  0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20
     12  13  14  15   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15   1   2   3   4   5
"""
function Mod_Poly(a,q) 
    period = q - 1
    if a isa Int
        if a < 0  # if a < 0, negative values follow a different rule from positive values
            while a < 0
                a = a + period
            end
            a = a + 1
        elseif a >= q  # if a >= q, positive values follow a different rule from negative values, (no need to process 0 to q - 1)
            while a >= q
                a = a - period
            end
        end
    else  # if a is a vector
        for i = 1:length(a)
            if a[i] < 0   # Same method
                while a[i] < 0
                    a[i] = a[i] + period
                end
                a[i] = a[i] + 1
            elseif a[i] >= q
                while a[i] >= q
                    a[i] = a[i] - period
                end
            end
        end
    end
    return a
end

Mod_Poly

In [88]:
#    ( +  *  / )  in F2^k


"""
    Add_Poly(a,b,Table)

    Addition in the field F2^k
"""
function Add_Poly(a,b,Table)
    @assert length(a) == length(b)
    if a isa Int  # If the input is integer                                    
        a = Index_to_Poly([a],Table)   #  Convert the corresponding index to polynomial, then apply the plus.
        b = Index_to_Poly([b],Table)   #  e.g. In F2^4, 2 = [0 1 0 0], 3 = [0 0 1 0], 2 + 3 = [0 1 0 0] + [0 0 1 0] = [0 1 1 0]
        Index = Poly_to_Index(mod.((a+b)[1],2),Table)  # Convert the polynomial back to index. e.g. [0 1 1 0] = 6
    else  # If the input is vector
        a = Index_to_Poly(a,Table)  # Convert the corresponding index to polynomial
        b = Index_to_Poly(b,Table)  # Since x and y are vectors, the result is in this form [[0 1 0 0],[0 0 1 0]]
        Index = zeros(Int64,length(a))
        for i = 1:length(a)
            Index[i] = Poly_to_Index(mod.(a[i]+b[i],2),Table)  # Convert the polynomial back to index.
        end
    end
    return Index
end



"""
    Mul_Poly(a,b,q)

    Multiplication in the field F2^k
"""
function Mul_Poly(a,b,q)
    if a == 0 || b == 0  # 0 * any = 0
        return 0
    elseif a isa Int && b isa Int  # integer * integer
        Index = Mod_Poly(Mod_Poly(a,q) + Mod_Poly(b,q) - 1, q)  # Multiplication of two polynomials is equivalent to the sum of their indexes - 1
        return Index
    elseif a isa Int && b isa Vector  # integer * vector
        Index = zeros(Int64,length(b))
        for i = 1:length(b)
            if b[i] == 0    # 0 * any = 0
                Index[i] = 0
            else
                Index[i] = Mod_Poly(Mod_Poly(a,q) + Mod_Poly(b[i],q) - 1, q)  # Multiplication of two polynomials is equivalent to the sum of their indexes - 1
            end
        end
        return Index
    end
end



"""
    Div_Poly(x,y,q)

    Division in the field F2^k
"""
function Div_Poly(a,b,q)
    @assert b != 0  # Cannot divide 0
    if a == 0    # 0 / any = 0
        return 0
    else
        return mod(Mod_Poly(a,q) - Mod_Poly(b,q), q - 1) + 1  # Division of two polynomials is equivalent to the division of their indexes + 1
    end
end

Div_Poly

In [89]:
#   a^k     vector a * vector b      fx/gx


"""
    Pow_Poly(x,power,q)

    Power in the field F2^k
"""
function Pow_Poly(a,power,q)
    if a == 0   # 0^any = 0
        return 0
    elseif power == 0   # any^0 = 1
        return 1
    elseif power > 0   # for positive power
        temp = a
        for i = 1:power-1
            a = Mul_Poly(a,temp,q)  # a^power
        end
        return a
    else   # for negative power
        temp = a
        for i = 1:power-1
            a = Mul_Poly(a,temp,q)  # a^power
        end 
        return Div_Poly(1,a,q)  # 1 / a^power = a^-power
    end
end



"""
    Conv_Poly(a,b,q)

    Convolution in the field F2^k
"""
function Conv_Poly(a,b,q)
    M = length(a) 
    N = length(b)
    R = zeros(Int64,M+N-1) # Initialize the result, its length is length(a) + length(b) - 1
    for i = 1:M
        for j = 1:N
            R[i+j-1] = Add_Poly(Mul_Poly(a[i],b[j],q),R[i+j-1],Table)  # R = a * b + R, this is calculated in the field F2^k
        end
    end
    return R
end



"""
    PolyDiv_Poly(fx,gx,q,Table) 

    Polynomial Division in the field F2^k
"""
function PolyDiv_Poly(fx,gx,q,Table) 
    fx = Mod_Poly(fx,q)  # Take the modulus of fx to ensure that coefficients of fx is in F2^k
    gx = Mod_Poly(gx,q)  # Take the modulus of gx to ensure that coefficients of gx is in F2^k
    deg_fx = getdeg(fx)  # get the degree of fx and gx
    deg_gx = getdeg(gx)

    fx = fx[1:deg_fx+1]  # Discard the extra 0 in fx, e.g. [1 2 1 0 0] -> [1 2 1]
    gx = gx[1:deg_gx+1]  # Discard the extra 0 in gx

    if gx == [0]
        return "Warning, cannot divide 0!"
    elseif deg_gx > deg_fx  # If the order of gx is higher than fx, the remainder is gx, the quotient is 0
        rx = fx
        qx = [0]
        return rem=rx,quo=qx
    else
        deg_diff = deg_fx - deg_gx  # degree difference between fx and gx
        qx = zeros(Int64,deg_diff+1)  # Initialize qx and rx
        rx = zeros(Int64,deg_gx+1)

        for i in 0:deg_diff
            # Add 0 to the gx vector to make gx and fx the same length
            # With the change of degree quotient, the position of gx is different each time
            zero_highdeg = zeros(Int64,i) 
            zero_lowdeg = zeros(Int64,deg_diff-i)
            gx_addzero = append!(zero_lowdeg,gx,zero_highdeg)

            # Calculate the quotient of each step
            # eg: Divide the highest order coefficient of fx by the highest order coefficient of gx for the first time
            qx[deg_diff+1-i] = Div_Poly(fx[deg_fx+1-i],gx[deg_gx+1],q)

            # Calculate the remainder, the remainder is the next divider
            temp = Mul_Poly(qx[deg_diff+1-i],gx_addzero,q)
            if temp != 0
                fx = Add_Poly(fx,temp,Table)
            end
        end
    end

    rx = convert(Vector{Int64},Mod_Poly(fx,q))  # the newest fx is the remainder, do modulation
    qx = convert(Vector{Int64},Mod_Poly(qx,q))

    return rx[1:getdeg(rx)+1],qx[1:getdeg(qx)+1]  # Discard extra 0s
end

PolyDiv_Poly

## 2. Generate message $m$ and codeword $c$


In [90]:
"""
    RSPoly_Enc(Table,k,a)

    Encode the message.
"""
function RSPoly_Enc(Table,k,a)
    # Generate the polynomial m(x)
    m = []  # message polynomial (coefficients)
    while length(m) < k
        push!(m,Table[rand(1:length(Table))])  # Randomly generate coefficients in the finite field for m
    end
    m = Poly_to_Index(m,Table)
    println("1.  Message m is ",m,"\n")

    # Generate codeword c
    q = length(Table)
    c = zeros(Int64,q-1)
    for n in 1:q-1
        for k in 1:length(m)
           c[n] = Add_Poly(Mul_Poly(m[k],Pow_Poly(a,(n-1)*(k-1),q),q),c[n],Table)
        end
    end
    c = Mod_Poly(c,q)
    println("2.  Codeword c is ",c,"\n")

    return m, c
end

RSPoly_Enc

In [91]:
q = 16
p = 2
f_x = [1,1,0,0,1] 
Table = Table_generator(q,p,f_x)
d = 5
n = q-1
k = n-d+1
a = 2  # a is chosen to be [0, 1, 0, 0] for simplification
RSPoly_Enc(Table,k,a)
nothing

1.  Message m is [8, 2, 5, 3, 9, 1, 10, 8, 8, 6, 0]

2.  Codeword c is [6, 7, 6, 2, 14, 11, 14, 6, 3, 3, 2, 3, 11, 4, 14]



## 3. Received word

The received word $r=c+e$ is got by introducing random errors in random positions to c.

In [92]:
using StatsBase


"""
    RSPoly_Channel(t,c,Table)

    Add noise to the codeword.
"""
function RSPoly_Channel(t,c,Table)
    Index = sample(collect(1:length(c)),t,replace=false); # Randomly choose t indexes using sample function, which is the position for errors
    println("3.  Error position is ",sort(Index,by=abs),"\n")  # show the sorted error position

    e = zeros(Int64,length(c))  # Initialize the error vector
    for i in 1:t
        e[Index[i]] = rand(1:length(Table)-1)   # Randomly generate the error 
    end
    r = Add_Poly(e,c,Table)  # Add the random errors to the codeword to form the received word, r = c + e

    println("4.  Error vector is ",e,"\n")
    println("5.  Received word is ",r,"\n")
    return r
end

RSPoly_Channel

In [93]:
q = 16
p = 2
f_x = [1,1,0,0,1] 
Table = Table_generator(q,p,f_x)
d = 5
n = q-1
k = n-d+1
t = floor(Int64,(d-1)/2)
a = 2  # a is chosen to be [0, 1, 0, 0] for simplification
m, c = RSPoly_Enc(Table,k,a)
RSPoly_Channel(t,c,Table)
nothing

1.  Message m is [6, 14, 10, 12, 9, 12, 4, 0, 11, 9, 3]

2.  Codeword c is [10, 7, 10, 6, 14, 2, 14, 5, 4, 15, 14, 0, 3, 3, 1]

3.  Error position is [9, 10]

4.  Error vector is [0, 0, 0, 0, 0, 0, 0, 0, 8, 7, 0, 0, 0, 0, 0]

5.  Received word is [10, 7, 10, 6, 14, 2, 14, 5, 5, 9, 14, 0, 3, 3, 1]



### 4. Generate $S\left(z\right)$ and $s$

The syndrome polynomial $S\left(z\right)$ can be generated as $S\left(z\right)=\sum_{j=0}^{n-k-1}{s_jz^j}$, where $s=rH^T$, $r$ is the received message, and $H$ is the parity-check matrix whose every entry equals to $\alpha^{i\left(j-1\right)}$ and size is $\left(n-k\right)\times n$.

In [94]:
using Polynomials


"""
    RSPoly_Syndrome(a,r,k,Table)

    Generate the Syndrome Polynomial
"""
function RSPoly_Syndrome(a,r,k,Table)
    q = length(Table)
    S = zeros(Int64,q-1-k)
    for i in 1:q-k-1
        for n in 1:q-1
           S[i] = Add_Poly(Mul_Poly(r[n],Pow_Poly(a,(n-1)*i,q),q),S[i],Table)  # Calculate S[i]
        end
    end
    S = Mod_Poly(S,q)
    println("6.  Syndrome polynomial S(z) = ",Polynomial(S,:z),"\n")  # show the syndrome polynomial
    return S  # return the coefficients of syndrome polynomial
end

RSPoly_Syndrome

In [95]:
q = 16
p = 2
f_x = [1,1,0,0,1] 
Table = Table_generator(q,p,f_x)
d = 5
n = q-1
k = n-d+1
t = floor(Int64,(d-1)/2)
a = 2  # a is chosen to be [0, 1, 0, 0] for simplification
m, c = RSPoly_Enc(Table,k,a)
r = RSPoly_Channel(t,c,Table)
RSPoly_Syndrome(a,r,k,Table)
nothing

1.  Message m is [8, 2, 12, 4, 14, 1, 12, 13, 12, 13, 10]

2.  Codeword c is [15, 11, 13, 2, 11, 7, 2, 9, 15, 6, 11, 10, 13, 7, 9]

3.  Error position is [5, 13]

4.  Error vector is [0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0]

5.  Received word is [15, 11, 13, 2, 4, 7, 2, 9, 15, 

6, 11, 10, 14, 7, 9]

6.  Syndrome polynomial S(z) = 13 + z + z^2 + 12*z^3



## 5. Generate $L\left(z\right)$ and $E\left(z\right)$

Apply  Euclidean algorithm to $z^{d-1}$ and $S\left(z\right)$

$$z^{d-1}=q_1\left(z\right)S\left(z\right)+r_1\left(z\right)$$
$$S\left(z\right)=q_2\left(z\right)r_1\left(z\right)+r_2\left(z\right)$$
$$...$$
$$r_{l-2}=q_l\left(z\right)r_{l-1}\left(z\right)+r_l\left(z\right)$$

where $deg(r_l\left(z\right))$≤ $t$-1

By Bezout's Identity, one has,
$$r_l\left(z\right)=a\left(z\right)S\left(z\right)+b\left(z\right)z^{d-1}\equiv a\left(z\right)S\left(z\right)mod {z^{d-1}}$$

Let $c$ be the leading coefficient of the polynomial $a\left(z\right)$, i.e., $c^{-1}a\left(z\right)$ is a monic polynomial. Then, $$L\left(z\right)=c^{-1}a\left(z\right)$$ $$E\left(z\right)=c^{-1}r_l\left(z\right)$$

In [96]:
"""
    find_E_and_L_Poly(Z,S,q,t,Table)

    Use Euclidian algorithm to find the remainder with the degree ≤ t-1 between fx and gx, and use Bezout's Identity to represent this remainder using fx and gx
"""
function find_E_and_L_Poly(Z,S,q,t,Table)
    rem,quo = PolyDiv_Poly(Z,S,q,Table)  # Polynomial division, Find the remainder and quotient of Z / S in p-domain
    if rem == [0] || getdeg(rem) <= (t-1)  # if the degree of remainder ≤ t-1 (use remainder == [0] because the function 'getdeg' cannot process [0])
        return rem, ones(Int64,1), Mod_Poly(quo,q)
    else
        rem,b_prev,a_prev = find_E_and_L_Poly(S,rem,q,t,Table)  # if not, iterate untill the degree of remainder ≤ t-1
        
        # Bezout's Identity
        b = Mod_Poly(a_prev,q)
        temp = Mod_Poly(Conv_Poly(a_prev,quo,q),q)
        a = Mod_Poly(Add_Poly(append!(b_prev,zeros(length(temp)-length(b_prev))),temp,Table),q)
        return rem,b,a  # remainder = b*Z + a*S = a*S mod z^(d-1) where deg(remainder) ≤ t-1
    end
end

find_E_and_L_Poly

In [97]:
using Polynomials


"""
    RSPoly_KeyPolys(S,q,d,t,Table)

    Find L and E use Euclidian algorithm to find the remainder with the degree ≤ t-1 between fx and gx, and use Bezout's Identity to represent this remainder using fx and gx
"""
function RSPoly_KeyPolys(S,q,d,t,Table)
    # Z = z^(d-1)
    Z = prepend!([1],zeros(Int64,d-1))
    
    E, ~, L = find_E_and_L_Poly(Z,S,q,t,Table)  # Apply Z/S, find E and L
    println("7.  Locator polynomial L(z) = ",Polynomial(L,:z),"\n")  # show the results
    println("8.  Error evaluator polynomial E(z) = ",Polynomial(E,:z),"\n")
    return E, L
end

RSPoly_KeyPolys

In [98]:
q = 16
p = 2
f_x = [1,1,0,0,1] 
Table = Table_generator(q,p,f_x)
d = 5
n = q-1
k = n-d+1
t = floor(Int64,(d-1)/2)
a = 2  # a is chosen to be [0, 1, 0, 0] for simplification
m, c = RSPoly_Enc(Table,k,a)
r = RSPoly_Channel(t,c,Table)
S = RSPoly_Syndrome(a,r,k,Table)
RSPoly_KeyPolys(S,q,d,t,Table)
nothing

1.  Message m is [1, 11, 5, 12, 8, 12, 6, 6, 9, 5, 15]

2.  Codeword c is [1, 2, 11, 3, 12, 14, 3, 1, 6, 8, 6, 14, 7, 2, 13]

3.  Error position is [6, 14]

4.  Error vector is [0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 13, 0]

5.  Received word is 

[1, 2, 11, 3, 12, 4, 3, 1, 6, 8, 6, 14, 7, 14, 13]

6.  Syndrome polynomial S(z) = 10 + 14*z + 15*z^2 + 12*z^3



7.  Locator polynomial L(z) = 10 + 2*z + 13*z^2

8.  Error evaluator polynomial E(z) = 4 + 7*z



## 6. Calculate estimated error $\hat{e}$ and estimated codeword $\hat{c}$
After getting $L(z)$ and $E(z)$, the error positions can be found from $L(z)$, and corresponding errors $e_i$ from $E(z)$.

Substitute $a^0,\alpha^{-1},\alpha^{-2},\cdots,\alpha^{-\left(p-2\right)}$ to $L(z)$, by this exhaustively computing, find all k that satisfy $L\left(\alpha^{-k}\right)=0$. The value of k represents error position.

As for the corresponding errors, $e_k=E\left(\alpha^{-k}\right)/\left(\alpha^k\mathrm{\Pi}_{j\neq k}\left(1-\alpha^j\alpha^{-k}\right)\right)=-E\left(\alpha^{-k}\right)/\frac{d}{dz}L\left(\alpha^{-k}\right)$. It is better to use the second one which has less complexity.

Then the codeword $\hat{c}$ can be recovered by $\hat{c}=r-\hat{e}$

In [99]:
"""
    Subs_Poly(x,fx,q,Table)

    Substitute x into f(x), return the result
"""
function Subs_Poly(x,fx,q,Table)
    L = length(fx)
    Result = zeros(Int64,L)
    for i in 1:L
        Result[i] = Mul_Poly(fx[i],Pow_Poly(x,i-1,q),q) # multiply each coefficient of fx with x^i-1
    end

    sum = 0
    for j in 1:L
        sum = Add_Poly(sum,Result[j],Table)  # sum all the elements of Result.
    end
    return sum
end



"""
    dfx_Poly(fx,q)

    Find the first derivative of fx
"""
function dfx_Poly(fx,q)
    dfx = zeros(Int64,length(fx)-1)    # Initialize the first derivative
    for i in 2:length(fx)
        dfx[i-1] = Mod_Poly(fx[i],q) * mod(i-1,2)  # Calculate each element of the first derivative.
    end
    return dfx
end



"""
    RSPol_Num_Err(L,E,q,r,Table)

    Estimate the error and recover the codeword
"""
function RSPol_Num_Err(L,E,q,r,Table)
    # Estimate the error position
    e_p = Int64[]   # Initialize the estimated error position
    for i in 1:q-1  # substitute 1, a^(-1), a^(-2), ..., a^(-(q-2)) to L
        if i == 1
            temp = 1
        else
            temp = Mod_Poly(-(i-1),q)
        end
        if Subs_Poly(temp,L,q,Table) == 0  # Substitute a^-(i-1) into L, check whether the result equals to 0
            push!(e_p,i)
        end
    end
    println("9. Estimated error position is ",e_p,"\n")

    # Estimate the error
    e_hat = zeros(Int64,q-1)  # Initialize the estimated error
    dL = dfx_Poly(L,q)  # the 1st derivative of L
    for j in 1:length(e_p)
        if e_p[j] == 1  # 1 is a particular case (inorder to distinguish 0 and a^0)
            #  e[k]       =               E[a^(-k)]        /        d[L(a^(-k))]/dz          in F2^q, -E[a^(-k)] = E[a^(-k)]
            e_hat[e_p[j]] = Div_Poly(Subs_Poly(1,E,q,Table),Subs_Poly(1,dL,q,Table),q)
        else
            e_hat[e_p[j]] = Div_Poly(Subs_Poly(Mod_Poly(-(e_p[j]-1),q),E,q,Table),Subs_Poly(Mod_Poly(-(e_p[j]-1),q),dL,q,Table),q)
        end
    end
    println("10. Estimated error is ", e_hat,"\n")

    # Estimate the codeword
    c_hat = Add_Poly(r,e_hat,Table)  # c = y + e     in F2^q, -e = e
    println("11. Estimated codeword is ", c_hat,"\n")
    return c_hat
end

RSPol_Num_Err

In [100]:
q = 16
p = 2
f_x = [1,1,0,0,1] 
Table = Table_generator(q,p,f_x)
d = 5
n = q-1
k = n-d+1
t = floor(Int64,(d-1)/2)
a = 2  # a is chosen to be [0, 1, 0, 0] for simplification
m, c = RSPoly_Enc(Table,k,a)
r = RSPoly_Channel(t,c,Table)
S = RSPoly_Syndrome(a,r,k,Table)
E,L = RSPoly_KeyPolys(S,q,d,t,Table)
RSPol_Num_Err(L,E,q,r,Table)
nothing

1.  Message m is [11, 1, 15, 12, 13, 3, 15, 3, 14, 6, 13]

2.  Codeword c is [5, 9, 2, 0, 7, 8, 13, 13, 12, 9, 12, 8, 9, 13, 0]

3.  Error position is [6, 7]

4.  Error vector is [0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0]

5.  Received word is [5, 9, 2, 0, 7, 13, 11,

 13, 12, 9, 12, 8, 9, 13, 0]

6.  Syndrome polynomial S(z) = 1 + 12*z + 4*z^2 + 3*z^3

7.  Locator polynomial L(z) = 9 + 3*z + 5*z^2

8.  Error evaluator polynomial E(z) = 9 + 11*z

9. Estimated error position is [6, 7]

10. Estimated error is [0,

 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0]

11. Estimated codeword is [5, 9, 2, 0, 7, 8, 13, 13, 12, 9, 12, 8, 9, 13, 0]



## 7. Recover the message 
With the estimated codeword $\hat{c}$, the estimated message $\hat{m}$ can be recovered. Since the original message $m$ is converted to $c$ by $m*G=c$, it can be converted back by $m=cG^{-1}$, hence $\hat{m}=\hat{c}G^{-1}$


For every entry of $G$, its value is $G\left[i,j\right]=\alpha^{\left(i-1\right)\left(j-1\right)}$ and it should be normallised in Galois field.

For example, in 8 filed, the $G$ should be

$$G=\left[\begin{matrix}1&1&1&1&1&1&1\\1&\alpha&\alpha^2&\alpha^3&\alpha^4&\alpha^5&\alpha^6\\1&\alpha^2&\alpha^4&\alpha^6&\alpha&\alpha^3&\alpha^5\\1&\alpha^3&\alpha^6&\alpha^2&\alpha^2&\alpha&\alpha^4\\\end{matrix}\right]$$

In [101]:
"""
    GaussElimination_poly(A,b,q,Table)

    Function for Gaussian Elimination
"""
function GaussElimination_poly(A,b,q,Table)
    L = LinearAlgebra.checksquare(A)  # Check whether matrix A is a square matrix, if true, return its size
    x = zeros(Int64,L,1)  # Initialize x, Ax = b, target is to solve x

    # Gaussian Elimination, transfer A to an upper-triangular matrix
    for i in 1:L-1
        for j in i+1:L
            a = Div_Poly(A[j,i],A[i,i],q)    # a = A[j,i]/A[i,i]
            for k in i+1:L
                A[j,k] = Add_Poly(A[j,k],Mul_Poly(A[i,k],a,q),Table)  # Multiply the ith row with a, use jth row to subtract it
            end
            A[j,i] = 0  # After multiplying the ith row with a, the ith element of the jth row becomes the same as the ith element of the ith row, hence the result of subtraction is 0

            b[j] = Add_Poly(b[j],Mul_Poly(b[i],a,q),Table)  #  Apply the same thing to b
        end
    end

    # Substitution, now we have an upper-triangular matrix A times x equals to b (A and b are different from the original A and b)
    x[L] = Div_Poly(b[L],A[L,L],q)  # Find x[L]

    # Substitute x[L], x[L-1],...,x[i+1] to calculate x[i]
    for i in L-1:-1:1
        sum = 0
        for j in i+1:L
            sum = Add_Poly(sum,Mul_Poly(A[i,j],x[j],q),Table)
        end
        x[i] = Div_Poly(Add_Poly(b[i],sum,Table),A[i,i],q)
    end
    return x
end

GaussElimination_poly

In [102]:
"""
    RSPoly_Message(q,k,a,c,Table)
 
    Function to recover message m from the codeword c
"""
function RSPoly_Message(q,k,a,c,Table)
    # Initialize the generator matrix
    G = zeros(Int64,k,q-2)  
    for i in 1:k
        for j in 1:q-2
            G[i,j] = Pow_Poly(a,(i-1)*(j-1),q)
        end
    end

    # Randomly choose k columns of the generator matrix G, make it become a square matrix
    Index = sort(sample(collect(1:q-2),k,replace=false),by=abs)

    # take the transpose since Gaussian Elimination should be in the form of Ax = b
    G_T = (G[:,Index])'

    # Choose the corresponding elements of c_hat
    c_hat = c[Index]

    # Use Gaussian Elimination to find the result, we have the function G_T * m_hat = c_hat (m and c are vectors, hence no need to take their transpose in code)
    m_hat = vec(GaussElimination_poly(G_T,c_hat,q,Table))
    println("12. Estimated message is ", m_hat,"\n")
    return m_hat
end

RSPoly_Message

In [103]:
q = 16
p = 2
f_x = [1,1,0,0,1] 
Table = Table_generator(q,p,f_x)
d = 5
n = q-1
k = n-d+1
t = floor(Int64,(d-1)/2)
a = 2  # a is chosen to be [0, 1, 0, 0] for simplification
m, c = RSPoly_Enc(Table,k,a)
r = RSPoly_Channel(t,c,Table)
S = RSPoly_Syndrome(a,r,k,Table)
E,L = RSPoly_KeyPolys(S,q,d,t,Table)
c_hat = RSPol_Num_Err(L,E,q,r,Table)
RSPoly_Message(q,k,a,c_hat,Table)
nothing

1.  Message m is [12, 15, 14, 14, 8, 2, 15, 13, 13, 6, 12]

2.  Codeword c is [13, 4, 12, 15, 14, 6, 11, 3, 6, 13, 9, 7, 7, 10, 0]

3.  Error position is [7, 11]

4.  Error vector is 

[0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 12, 0, 0, 0, 0]

5.  Received word is [13, 4, 12, 15, 14, 6, 5, 3, 6, 13, 8, 7, 7, 10, 0]

6.  Syndrome polynomial S(z) = 15 + 8*z + 4*z^2 + 2*z^3

7.  Locator polynomial L(z) = 3 + 10*z + 4*z^2

8.  Error evaluator polynomial E(z) = 2 + 13*z

9. Estimated error position is [7, 11]

10. Estimated error is [0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 12, 0, 0, 0, 0

]

11. Estimated codeword is [13, 4, 12, 15, 14, 6, 11, 3, 6, 13, 9, 7, 7, 10, 0]



12. Estimated message is [12, 15, 14, 14, 8, 2, 15, 13, 13, 6, 12]



## 8.  Decoding process
Implement the procedures above to complete the whole decoding process.

In [104]:
"""
    RSPoly_Dec(q,d,t,a,r,k,Table)

    Decoding function, combine steps 4-7
"""
function RSPoly_Dec(q,d,t,a,r,k,Table)
    S = RSPoly_Syndrome(a,r,k,Table)
    E,L = RSPoly_KeyPolys(S,q,d,t,Table)
    c_hat = RSPol_Num_Err(L,E,q,r,Table)
    m_hat = RSPoly_Message(q,k,a,c_hat,Table)
    return m_hat
end

RSPoly_Dec

In [105]:
q = 16
p = 2
f_x = [1,1,0,0,1] 
Table = Table_generator(q,p,f_x)
d = 5
n = q-1
k = n-d+1
t = floor(Int64,(d-1)/2)
a = 2  # a is chosen to be [0, 1, 0, 0] for simplification
m, c = RSPoly_Enc(Table,k,a)
r = RSPoly_Channel(t,c,Table)
RSPoly_Dec(q,d,t,a,r,k,Table)
nothing

1.  Message m is [4, 12, 11, 1, 2, 3, 12, 0, 1, 15, 9]

2.  Codeword c is [9, 1, 8, 4, 14, 5, 0, 5, 1, 15, 4, 0, 14, 7, 5]

3.  Error position is [3, 9]

4.  Error vector is [0, 0, 14, 0, 0, 0, 0, 0, 14, 0, 0, 0, 0, 0, 0]

5.  Received word is [9, 1, 6, 4, 14, 5, 0, 5, 7, 15, 4, 0, 14, 7, 5]



6.  Syndrome polynomial S(z) = 14 + 14*z + 4*z^2 + 14*z^3

7.  Locator polynomial L(z) = 1 + z + 11*z^2

8.  Error evaluator polynomial E(z) = 14

9. Estimated error position is [3, 9]

10. Estimated error is [0, 0, 14, 0, 0, 0, 0, 0, 14, 0, 0, 0, 0, 0, 0]

11. Estimated codeword is [9, 1, 8, 4, 14, 5, 0, 5, 1, 15, 4, 0, 14, 7, 5]

12. Estimated message is [4, 12, 11, 1, 2, 3, 12, 0, 1, 15, 9]



## 9.  Final Function
The final combined function.

In [106]:
"""
    RSPoly(q,p,d,n,k,t,a,Table)

    Final function
"""
function RSPoly(q,p,d,n,k,t,a,Table)
    if isprime(p) == false
        println("p is not a prime number. Please change the value of p.\n")
        return "Warning"
    end

    # Generate the message and codeword
    m, c = RSPoly_Enc(Table,k,a)

    # Generate noise and add this noise to the codeword
    r = RSPoly_Channel(t,c,Table)

    # Recover the message
    m_hat = RSPoly_Dec(q,d,t,a,r,k,Table)

    # Check the correctness
    if m == m_hat
        println("🙂🙂🙂 Recovery Successful 🙂🙂🙂\n")
        flag = true
    else
        println("!!!!! Fail to recover message m !!!!!\n")
        flag = false
    end

    println("Put original message and estimated message together to compare:")
    println(m)
    println(m_hat)
    println("\nThe above m is in the Index form, the polynomial form of m is:")
    println(Index_to_Poly(m_hat,Table))
    println("\nFor convenience, the table is provided:")
    println(Table)
    return flag
end

RSPoly

In [107]:
q = 16
p = 2
f_x = [1,1,0,0,1] 
Table = Table_generator(q,p,f_x)
d = 5
n = q-1
k = n-d+1
t = floor(Int64,(d-1)/2)
a = 2  # a is chosen to be [0, 1, 0, 0] for simplification
RSPoly(q,p,d,n,k,t,a,Table)
nothing

1.  Message m is [8, 4, 15, 11, 11, 11, 12, 11, 2, 4, 9]

2.  Codeword c is 

[8, 2, 5, 6, 6, 0, 0, 0, 1, 11, 1, 13, 5, 11, 14]

3.  Error position is [6, 15]

4.  Error vector is [0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 2]

5.  Received word is [8, 2, 5, 6, 6, 7, 0, 0, 1, 11, 1, 13, 5, 11, 13]

6.  Syndrome polynomial S(z) = 13 + 8*z + z^2 + z^3

7.  Locator polynomial L(z) = 3 + 15*z + 7*

z^2

8.  Error evaluator polynomial E(z) = 15 + 3*z

9. Estimated error position is [6, 15]

10. Estimated error is [0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 2]

11. Estimated codeword is [8, 2, 5, 6, 6, 0, 0, 0, 1, 11, 1, 13, 5, 11, 14]

12. Estimated message is [8, 4, 15, 11, 11, 11, 12, 11, 2, 4, 9]

🙂🙂🙂 Recovery Successful 🙂🙂🙂

Put original message and estimated message together to compare:
[8, 4, 15, 11, 11, 11, 12, 11, 2, 4, 9]
[8, 4, 15, 11, 11, 11, 12, 11, 2, 4, 9]

The above m is in the Index form, the polynomial form of m is:
Any[[1, 1, 0, 1], [0, 0, 0, 1], [1, 0, 0, 1], 

[1, 1, 1, 0], [1, 1, 1, 0], [1, 1, 1, 0], [0, 1, 1, 1], [1, 1, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [1, 0, 1, 0]]

For convenience, the table is provided:
Any[[0, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [

0, 0, 1, 0], [0, 0, 0, 1], [1, 1, 0, 0], [0, 1, 1, 0], [0, 0, 1, 1], [1, 1, 0, 1], [1, 0, 1, 0], [0, 1, 0, 1], [1, 1, 1, 0], [0, 1, 1, 1], [1, 1, 1, 1], [1, 0, 1, 1], [1, 0, 0, 1]]


### 3.2.2 RS Codes on $\mathbb{F}_{2^k}$

The above code should work for other choices of $k$ in $\mathbb{F}_{2^k}$. In 3.2.1, we have worked on the case where $k=4$. Try a different $k$ with $5 \le k \le 10$. 

This is the same as 3.2.1.

In [108]:
# Test
q = 64
p = 2
f_x = [1, 1, 1, 0, 0, 1, 1]
Table = Table_generator(q,p,f_x)
d = 7
n = q-1
k = n-d+1
t = floor(Int64,(d-1)/2)
a = 2  # a is chosen to be [0, 1, 0, 0] for simplification
RSPoly(q,p,d,n,k,t,a,Table)
nothing

1.  Message m is [41, 29, 16, 57, 39, 40, 21, 0, 50, 25, 12, 41, 32, 48, 45, 1, 37, 11, 36, 12, 45, 46, 28, 8, 59, 42, 1, 36, 2, 7, 15, 31, 13, 34, 48, 9, 23, 46, 3, 31, 62, 6, 41, 35, 25, 2, 41, 21, 57, 38, 2, 16, 15, 18, 36, 5, 52]

2.  Codeword c is [35, 32, 30, 24, 55, 16, 54, 56

, 40, 57, 57, 10, 21, 6, 18, 32, 12, 14, 23, 41, 18, 1, 48, 63, 62, 33, 20, 9, 15, 46, 47, 25, 9, 28, 11, 9, 44, 63, 4, 4, 24, 15, 7, 11, 50, 60, 25, 13, 50, 39, 9, 63, 15, 49, 0, 3, 4, 61, 9, 46, 53, 19, 2]

3.  Error position is [9, 36, 44]

4.  Error vector is [0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 

0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

5.  Received word is [35, 32, 30, 24, 55, 16, 54, 56, 49, 57, 57, 10, 21, 6, 18, 32, 12, 14, 23, 41, 18, 1, 48, 63, 62, 33, 20, 9, 15, 46, 47, 25, 9, 28, 11, 22, 44, 63, 4, 4, 24, 15, 7, 51, 50, 60, 25, 13, 50, 39, 9, 63, 15, 49, 0, 3, 4, 61, 9, 46, 53, 19, 2]

6.  Syndrome polynomial S(z) = 12 + 36*z + 17*z^2 + 61*z^3 + 61*z^4 + 37*z^5

7.  Locator polynomial L(z) = 3 + 50*z + 56*z^2 + 26*z^3

8.  Error evaluator polynomial E(z) = 14 + 3*z + 57*z^2

9. Estimated error position is [9, 36, 44]

10. Estimated error is [0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

11. Estimated codeword is [35, 32, 30, 24, 55, 16, 54, 56, 40, 57, 57, 10, 21, 6, 18, 32, 12, 14, 23, 41, 18, 1, 48, 63, 62, 33, 20, 9, 15, 46, 47, 25, 9, 28, 11, 9, 44, 63, 4, 4, 24, 15, 7, 11, 50, 60,

, 53, 19, 2]



12. Estimated message is [41, 29, 16, 57, 39, 40, 21, 0, 50, 25, 12, 41, 32, 48, 45, 1, 37, 11, 36, 12, 45, 46, 28, 8, 59, 42, 1, 36, 2, 7, 15, 31, 13, 34, 48, 9, 23, 46, 3, 31, 62, 6, 41, 35, 25, 2, 41, 21, 57, 38, 2, 16, 15, 18, 36, 5, 52]

🙂🙂🙂 Recovery Successful 🙂🙂🙂

Put original message and estimated message together to compare:
[41, 29, 16, 57, 39, 40, 21, 0, 50, 25, 12, 41, 32, 48, 45, 1, 37, 11, 36, 12, 45, 46, 28, 8, 59, 42, 1, 36, 2, 7, 15, 31, 13, 34, 48, 9, 23, 46, 3

...

 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1], [1, 1, 1, 0, 0, 1], [1, 0, 0, 1, 0, 1], [1, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0], [0, 1, 0, 1, 1, 0], [0, 0, 1, 0, 1, 1], 

[1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 0], [0, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0], [0, 1, 1, 1, 1, 1], [1, 1, 0, 1, 1, 0], [0, 1, 1, 0, 1, 1], [1, 1, 0, 1, 0, 0], [0, 1, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1], [1, 1, 1

, 1, 1, 1], [1, 0, 0, 1, 1, 0], [0, 1, 0, 0, 1, 1], [1, 1, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0], [0, 0, 1, 1, 0, 0], [0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 1, 1], [1, 1, 1, 0, 0, 0], [0, 1, 1, 1, 0, 0], [0, 0,

 1, 1, 1, 0], [0, 0, 0, 1, 1, 1], [1, 1, 1, 0, 1, 0], [0, 1, 1, 1, 0, 1], [1, 1, 0, 1, 1, 1], [1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 1], [1, 1, 0, 0, 0, 1], [1, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 1], [1, 0, 1, 1, 0, 1], [1, 0, 1, 1, 1, 1], [1, 0, 1, 1, 1, 0], [0, 1, 0, 1, 1, 1], [1, 1, 0, 0, 1, 0], [0, 1, 1, 0, 0, 1], [1, 1, 0, 1, 0, 1], [1, 0, 0, 0, 1, 1], [1, 0, 1, 0, 0, 0], [0, 1, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0], [0, 0, 0, 1, 0, 1], 

[1, 1, 1, 0, 1, 1], [1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1], [1, 1, 1, 1, 0, 1], [1, 0, 0, 1, 1, 1], [1, 0, 1, 0, 1, 0], [0, 1, 0, 1, 0, 1], [1, 1, 0, 0, 1, 1]]


In [109]:
# Test
q = 128
p = 2
f_x = [1, 0, 0, 0, 0, 0, 1, 1]
Table = Table_generator(q,p,f_x)
d = 7
n = q-1
k = n-d+1
t = floor(Int64,(d-1)/2)
a = 2  # a is chosen to be [0, 1, 0, 0] for simplification
RSPoly(q,p,d,n,k,t,a,Table)
nothing

1.  Message m is [104, 58, 84, 66, 14, 127, 67, 82, 90, 10, 49, 28, 25, 50, 5, 58, 6, 27, 35, 30, 0, 90, 10, 74, 37, 31, 46, 10, 60, 83, 48, 125, 80, 1, 127, 8, 125, 85, 102, 37, 20, 41, 30, 49, 58, 40, 109, 69, 23, 65, 111, 56, 64, 125, 27, 57, 126, 90, 82, 79, 34, 127, 57, 125, 104, 71, 108, 47, 82, 105, 80, 27, 116, 127, 126, 73, 0, 116, 45, 1, 4, 66, 28, 19, 111, 69, 45, 2, 89, 51, 126, 26, 2, 17, 44, 104, 32, 8, 127, 75, 109, 8, 2, 55, 74, 45, 0, 88, 121, 72, 84, 20, 51, 77, 54, 55, 8, 72, 105, 105, 54]



2.  Codeword c is [34, 120, 0, 53, 38, 9, 46, 114, 64, 70, 29, 119, 74, 43, 36, 57, 102, 9, 45, 2, 37, 60, 22, 50, 113, 0, 70, 115, 4, 113, 16, 71, 106, 12, 57, 32, 101, 74, 115, 32, 79, 109, 28, 24, 8, 106, 81, 94, 21, 66, 99, 81, 2, 103, 72, 122, 100, 0, 31, 25, 108, 62, 82, 77, 115, 62, 1, 30, 40, 22, 8, 96, 62, 124, 11, 105, 54, 100, 22, 89, 58, 89, 1, 100, 121, 49, 56, 40, 35, 93, 68, 98, 65, 121, 82, 108, 60, 21, 83, 98, 88, 78, 57, 119, 65, 115, 77, 104, 103, 127, 85, 117, 15, 10, 120, 126, 79, 93, 13, 97, 123, 54, 104, 37, 92, 0, 71]

3.  Error position is [44, 54, 63]

4.  Error vector is [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 120, 0, 0, 0, 0, 0, 0, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

5.  Received word is [34, 120, 0, 53, 38, 9, 46, 114, 64, 70, 29, 119, 74, 43, 36, 57, 102, 9, 45, 2, 37, 60, 22, 50, 113, 0, 70, 115, 4, 113, 16, 71, 106, 12, 57, 32, 101, 74, 115, 32, 79, 109, 28, 9, 8, 106, 81, 94, 21, 66, 99, 81, 2, 28, 72, 122, 100, 0, 31, 25, 108, 62, 15, 77, 115, 62, 1, 30, 40, 22, 8, 96, 62, 124, 11, 105, 54, 100, 22, 89, 58, 89, 1, 100, 121, 49, 56, 40, 35, 93, 68, 98, 65, 121, 82, 108, 60, 21, 83, 98, 88, 78, 57, 119, 65, 115, 77, 104, 103, 127, 85, 117, 15, 10, 120, 126, 79, 93, 13, 97, 123, 54, 104, 37, 92, 0, 71]

6.  Syndrome polynomial S(z) = 101 + 22*z + 43*z^2 + 73*z^3 + 79*z^4 + 13*z^5

7.  Locator polynomial L(z) 

 + 101*z^2

9. Estimated error position is [44, 54, 63]

10. Estimated error is [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 120, 0, 0, 0, 0, 0, 0, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

11. Estimated codeword is [34, 120, 0, 53, 38, 9, 46, 114, 64, 70, 29, 119, 74, 43, 36, 57, 102, 9, 45, 2, 37, 60, 22, 50, 113, 0, 70, 115, 4, 113, 16, 71, 106, 12, 57, 32, 101, 74, 115, 32, 79, 109, 28, 24, 8, 106, 81, 94, 21, 66, 99, 81, 2, 103, 72, 122, 100, 0, 31, 25, 108, 62, 82, 77, 115, 62, 1, 30, 40, 22, 8, 96, 62, 124, 11, 105, 54, 100, 22, 89, 58, 89, 1, 100, 121, 49, 56, 40, 35, 93, 68, 98, 65, 121, 82, 108, 60, 21, 83, 98, 88, 78, 57, 119, 65, 115, 77, 104, 103, 127, 85, 117, 15, 10, 120, 126, 79, 93, 13, 97

, 123, 54, 104, 37, 92, 0, 71]



...

 1, 0], [0, 1, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 0, 1], [1, 1, 0, 1, 0, 0, 1], [1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 1, 0, 1, 1], [1, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0], [0, 0, 1, 1, 1, 1, 1]

, [1, 0, 0, 1, 1, 1, 0], [0, 1, 0, 0, 1, 1, 1], [1, 0, 1, 0, 0, 1, 0], [0, 1, 0, 1, 0, 0, 1], [1, 0, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 1], [1, 1, 1, 0, 1, 0, 0], [0, 1, 1, 1, 0, 1, 0], 

[0, 0, 1, 1, 1, 0, 1], [1, 0, 0, 1, 1, 1, 1], [1, 1, 0, 0, 1, 1, 0], [0, 1, 1, 0, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0], [0, 1, 0, 1, 1, 0, 0], [0, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 1], [1, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 1], [1, 0, 0, 1, 0, 0, 1], [1, 1, 0, 0, 1, 0, 1], [1, 1, 1, 0, 0, 1, 1], [1

, 1, 1, 1, 0, 0, 0], [0, 1, 1, 1, 1, 0, 0], [0, 0, 1, 1, 1, 1, 0], [0, 0, 0, 1, 1, 1, 1], [1, 0, 0, 0, 1, 1, 0], [0, 1, 0, 0, 0, 1, 1], [1, 0, 1, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0], [0, 0, 0, 1,

 0, 1, 0], [0, 0, 0, 0, 1, 0, 1], [1, 0, 0, 0, 0, 1, 1], [1, 1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0], [0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 1, 1]]


In [110]:
# Test
# This result should be fail !!!!!!!!!!!!!!


q = 32
p = 2
f_x = [1, 0, 0, 1, 0, 1]
Table = Table_generator(q,p,f_x)
d = 7
n = q-1
k = n-d+1
t = floor(Int64,(d-1)/2) + 1     # If t exceed its limitation 
a = 2  # a is chosen to be [0, 1, 0, 0] for simplification
RSPoly(q,p,d,n,k,t,a,Table)
nothing

1.  Message m is [11, 7, 13, 7, 23, 6, 1, 29, 17, 28, 10, 22, 2, 1, 12, 2, 19, 28, 12, 23, 31, 30, 25, 10, 4]

2.  Codeword c is [6, 10, 11, 6, 13, 7, 29, 9, 23, 26, 15, 23, 19, 15, 4, 19, 5, 2, 13, 23, 14, 9, 10, 19, 19, 12, 26, 11, 14, 11, 29]

3.  Error position is [3, 9, 13, 30]

4.  Error vector is [0, 0, 8, 0, 0, 0, 0, 0, 17, 0, 0, 0, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 0]

5.  Received word is [6, 10, 13, 6, 13, 7, 29, 9, 27, 26, 15, 23, 8, 15, 4, 19, 5, 2, 13, 23, 14, 9, 10, 19, 19, 12, 26, 11, 14, 12, 29]

6.  Syndrome polynomial S(z) = 6 + 25*z + z^2 + 13*z^3 + 12*z^4 + 25*z^5

7.  Locator polynomial L(z) = 13 + 9*z + 26*z^2

8.  Error evaluator polynomial E(z) = 18 + 25*z + 4*z^2 + 21*z^3



9. Estimated error position is Int64[]

10. Estimated error is [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

11. Estimated codeword is [6, 10, 13, 6, 13, 7, 29, 9, 27, 26, 15, 23, 8, 15, 4, 19, 5, 2, 13, 23, 14, 9, 10, 19, 19, 12, 26, 11, 14, 12, 29]

12. Estimated message is [25, 18, 26, 15, 21, 8, 4, 30, 1, 31, 21, 8, 1, 28, 19, 21, 25, 1, 21, 13, 3, 3, 14, 13, 1]

!!!!! Fail to recover message m !!!!!

Put original message and estimated message together to compare:
[11, 7, 13, 7, 23, 6, 1, 29, 17

, 28, 10, 22, 2, 1, 12, 2, 19, 28, 12, 23, 31, 30, 25, 10, 4]
[25, 18, 26, 15, 21, 8, 4, 30, 1, 31, 21, 8, 1, 28, 19, 21, 25, 1, 21, 13, 3, 3, 14, 13, 1]

The above m is in the Index form, the polynomial form of m is:
Any[[0, 0, 1, 1, 1], [0, 0, 0, 1, 1], [1, 0, 0, 0, 1], [1, 1, 0, 0, 0], [1, 

1, 1, 1, 1], [1, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 1, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 1], [1, 1, 1, 1, 1], [1, 0, 1, 1, 0], [1, 0, 0, 0, 0], [0, 1

, 1, 0, 1], [1, 0, 0, 1, 1], [1, 1, 1, 1, 1], [0, 0, 1, 1, 1], [1, 0, 0, 0, 0], [1, 1, 1, 1, 1], [0, 1, 1, 1, 1], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 0, 1, 0

, 1], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0]]

For convenience, the table is provided:
Any[[0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], 

[0, 0, 0, 0, 1], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0], [0, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 0, 1], [1, 1, 1, 1, 0], [0, 1, 1, 1, 1], [1, 0, 1, 0, 1], 

[1, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 1], [1, 0, 0, 1, 1], [1, 1, 0, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 0, 1], 

[1, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 1, 1], [1, 0, 0, 0, 1], [1, 1, 0, 1, 0], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0], [0, 1, 0, 1, 0], [0, 0, 1, 0, 1]]


## Highlight

Please list a couple of highlights of your coursework that may impress your markers.

1. We provide multiple versions of functions to some questions, although some solution has problems, they are usually caused by the inaccuracy of floating point numbers, and we still find nice ways to solve all the questions.


2. We almost explain everything in detail, with enough annotations.


3. We implement some nice approaches to difficult questions, such as the Gaussian elimination.


4. In 3.2, we map the polynomials to Indexes, this allows us to implement the whole process almost as usual --- 1. map the polynomials to Index, 2. Use Indexes for calculation, 3. Convert Indexes back to polynomials. The only difficulty is the arithmetical operation between Indexes, they are different from the normal operations. Hence, we designed our own addition, multiplication, division, modulation, etc. The result shows that our idea is correct.